## load libraries

In [ ]:
# load libraries
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library("tidyverse")
# quiet_library("hise")
quiet_library("ArchR")
quiet_library("data.table")
quiet_library("jsonlite")
quiet_library("parallel")
quiet_library("Seurat")
# Load scMACS and accompanying libraries
quiet_library("MOCHA")
quiet_library("GenomicRanges")
quiet_library("plyranges")
quiet_library("ggplot2")
quiet_library("SummarizedExperiment")


In [ ]:
# # import MP's function to do label transfer from atac to atac
# source('/home/jupyter/github/scATAC_Supplements/scATAC_CellTypeLabeling.R')
suppressPackageStartupMessages(source('/home/jupyter/github/scATAC_Supplements/ArchR_Supplements.R'))
# source('/home/jupyter/github/Teaseq-analysis/scRNA_teaseq_ananlysis_helper_functions.r')

In [ ]:
#Load scMACS and accompanying libraries
library(MOCHA)
library(data.table)
library(ArchR)
library(GenomicRanges)
library(plyranges)

In [ ]:
package.version('MOCHA')

In [ ]:
# Define your annotation package for TxDb object(s)
# and genome-wide annotation
# Here our samples are human using hg38 as a reference.
# For more info: https://bioconductor.org/packages/3.15/data/annotation/
library(TxDb.Hsapiens.UCSC.hg38.refGene)
library(org.Hs.eg.db)
TxDb <- TxDb.Hsapiens.UCSC.hg38.refGene
Org <- org.Hs.eg.db

In [ ]:
# define work directories
proj_path <- '/home/jupyter/data/preRA_teaseq/EXP-00243'
data_path <- '/home/jupyter/data/preRA_teaseq/EXP-00243'
setwd(proj_path)
# define a project name
proj_name <- 'preRA_teaseq'
fig_path <- as.character('/home/jupyter/figures/preRA_teaseq/ATAC')
if(!dir.exists(fig_path)) (dir.create(fig_path, recursive = TRUE))

output_path <- '/home/jupyter/data/preRA_teaseq/output_results/atac'
if(!dir.exists(output_path)) (dir.create(fig_path, recursive = TRUE))


In [ ]:
# set up the parallel computing parameters
# set up future for seurat
# Check number of cores
future::availableCores()
# Set up parallel processing to run when using 'future' functions 
future::plan(strategy = "multicore", workers = future::availableCores()-3)  
options(future.globals.maxSize = 1000 * 1024^5)

# set ArchR parameters
addArchRThreads(threads = 58)
addArchRGenome("hg38")
set.seed(1221)
options(repr.plot.width = 20, repr.plot.height = 15)

In [ ]:
# Define your annotation package for TxDb object(s)
# and genome-wide annotation
# Here our samples are human using hg38 as a reference.
# For more info: https://bioconductor.org/packages/3.15/data/annotation/
library(TxDb.Hsapiens.UCSC.hg38.refGene)
library(org.Hs.eg.db)
TxDb <- TxDb.Hsapiens.UCSC.hg38.refGene
Org <- org.Hs.eg.db

In [ ]:
# define the color palette to be used
npg_color <- c("#E64B35FF", "#4DBBD5FF", "#00A087FF", "#3C5488FF", "#F39B7FFF", 
               "#8491B4FF", "#91D1C2FF", "#DC0000FF", "#7E6148FF", "#B09C85FF")
nejm_color <- c("#BC3C29FF", "#0072B5FF", "#E18727FF", "#20854EFF", "#7876B1FF", "#6F99ADFF", "#FFDC91FF", "#EE4C97FF")
jama_color <- c("#374E55FF", "#DF8F44FF", "#00A1D5FF", "#B24745FF", "#79AF97FF", "#6A6599FF", "#80796BFF")
jco_color <- c("#0073C2FF", "#EFC000FF", "#868686FF", "#CD534CFF", "#7AA6DCFF", "#003C67FF", "#8F7700FF")
cluster_colors <- c("#DC050C", "#FB8072", "#1965B0", "#7BAFDE", "#882E72", "#B17BA6", "#FF7F00", "#FDB462", "#E7298A", 
    "#E78AC3", "#33A02C", "#B2DF8A", "#55A1B1", "#8DD3C7", "#A6761D", "#E6AB02", "#7570B3", "#BEAED4", "#666666", "#999999", 
    "#aa8282", "#d4b7b7", "#8600bf", "#ba5ce3", "#808000", "#aeae5c", "#1e90ff", "#00bfff", "#56ff0d", "#ffff00")
cluster_colors_ext <- colorRampPalette(cluster_colors)(36)
options(repr.plot.width = 20, repr.plot.height = 15)

In [ ]:
# helper function

#### add metadata from seurat object to archr project
## extract the columns from seurat to add to archr
## add metadata from seurat to archr
#' @param archr: archr project to add metadata on
#' @param so: seurat object to extract metadata from
#' @param so.cols: a list of column names in the seurat obeject
#' @param id.col: column name of the barcodes which links cells in seurat and archr object, default setting is barcode

#' @return archr a SummarizedExperiment object which nomarlized counts are aggregrate

#' @examples 
# ra_mye_atac <- MetaSotoArchr(archr=ra_mye_atac, so=ra_so_mye, 
#                            so.cols=c('barcodes', 'subject_id', 'cohort', 
#                                      'total_pbmc_counts','Tiles_snn_res.0.8', 'wsnn_res.0.8'), 
#                           id.col='barcodes')

MetaSotoArchr <- function(archr, so, so.cols, id.col='barcodes', save.archr=TRUE){
    # check if all the so.cols are in the seurat metadata
    if(!all(so.cols %in% colnames(so@meta.data))){
        stop('Not all the columns are present in the seurat object')
    }
    if(any(so.cols[so.cols!=id.col] %in% colnames(getCellColData(archr)))){
        warning('the column names to add are already in the archr object, this will create suffix to the columns')
    }
    seurat_metadata <- so@meta.data %>% 
        select(all_of(c(so.cols, id.col))) %>% as_tibble()
    # check if id coloum are match between seurat and archr
    so_ids <- so@meta.data %>% pull(.data[[id.col]]) 
    archr_ids <- getCellColData(archr) %>% as_tibble() %>% pull(.data[[id.col]]) 
    if(!setequal(so_ids, archr_ids)){
        stop('id col in the seurat and archr object dont match.')
    }
    # add metadata from seurat to archR
    cell_id <- archr@cellColData %>% rownames()
    archr@cellColData <- merge(archr@cellColData, seurat_metadata, by = id.col, sort=FALSE)
    rownames(archr@cellColData) <- cell_id    
    if(save.archr){
        saveArchRProject(ArchRProj = archr)
    }
    return(archr)
}

##This function extracts the RNA counts from a Seurat object and adds them to an ArchR Project. 
##ATAC_Cell_ID is the metadata column of the Seurat object that contains the matching cell ids to the ArchR Project. Make sure these align before you do anything. 
##You can set force to be TRUE or FALSE, depending on whether you want to 
##over-write the existing GeneExpressionMatrix within the ArchR Project.
##Requires: plyranges, dplyr, Seurat, and ArchR
addSeuratRNA <- function(SeuratObject, ArchRProject, ATAC_Cell_ID, assay = "RNA", force1 = FALSE){

	counts1 <- as.SingleCellExperiment(SeuratObject, assay = assay)
	colnames(counts1) = SeuratObject@meta.data[,ATAC_Cell_ID]
	genes <- getGenes(ArchRProject) %>% plyranges::filter(symbol %in% as.character(rownames(counts1))) %>% 
		arrange(match(symbol,as.character(rownames(counts1))))
	counts2 <-counts1[rownames(counts1) %in% genes$symbol]
	counts3 <- SummarizedExperiment(assays=list(counts=assays(counts2)[[1]]),
		 rowRanges= genes) 
	ArchRProject <- addGeneExpressionMatrix(ArchRProject,
					seRNA = counts3, force = force1)
	return(ArchRProject)
}

## load archR for all cells

In [ ]:
# # load archR data
# ra_atac <- loadArchRProject(path = "/home/jupyter/data/preRA_teaseq/EXP-00243/atac_arrows")
# ra_atac

## remove BR2024 from the analysis

In [ ]:
# # remove the sample BR2024
# idxSample <- BiocGenerics::which(ra_atac$subject_id != "BR2024")
# cellsSample <- ra_atac$cellNames[idxSample]
# ra_atac_fl <- subsetArchRProject(ra_atac, cells=cellsSample)

In [ ]:
# saveArchRProject(ra_atac_fl, outputDirectory = "/home/jupyter/data/preRA_teaseq/EXP-00243/ArchRSubset")

In [ ]:
# load archR data
ra_atac_fl <- loadArchRProject(path = "/home/jupyter/data/preRA_teaseq/EXP-00243/ArchRSubset")
ra_atac_fl

In [ ]:
ra_atac_fl$subject_id %>% unique()

In [ ]:
# merge suba and suba monocyte
ra_atac_fl$clean_l2_cell_types <- if_else(ra_atac_fl$clean_l2_cell_types %in% 
                                       c('cd14mono_sub_a', 'cd14mono_sub_b', 'cd14mono_other'), 
                                       'cd14mono', ra_atac_fl$clean_l2_cell_types)

In [ ]:
ra_atac_fl$clean_l2_cell_types%>%unique()

In [ ]:
ra_atac_fl <- addIterativeLSI(
  ArchRProj = ra_atac_fl,
  useMatrix = "TileMatrix", 
  name = "IterativeLSI", 
  iterations = 2,
  varFeatures = 75000, # increase the viable features
  force = TRUE
)

ra_atac_fl <- addClusters(
  input = ra_atac_fl,
  reducedDims = "IterativeLSI",
  method = "Seurat",
  name = "Clusters",
  resolution = 3,
  force = TRUE
)

ra_atac_fl <- addUMAP(ArchRProj = ra_atac_fl, 
                    reducedDims = "IterativeLSI", force = TRUE)

In [ ]:
saveArchRProject(ra_atac_fl)

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(ra_atac_fl, embedding = "UMAP", 
                    colorBy = "cellColData", name = c('Clusters', "l1_cell_types", "clean_l2_cell_types",
                                                     "cohort",  "subject_id"))
p1
# p2 <- plotEmbedding(ra_atac, embedding = "UMAP", 
#                     colorBy = "GeneExpressionMatrix", name = c('PLCG2', 'JUNB', 'HLA-E'))
plotPDF(p1, name = paste0(proj_name, '_all_cells_annotated_atac_umap.pdf'), ArchRProj = ra_atac_fl,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
# export LSI matrix
getAvailableMatrices(ra_atac_fl)

In [ ]:
# export LSI matrix
lsi_matrix <- getReducedDims(ra_atac_fl, reducedDims = "IterativeLSI")%>%as_tibble()
lsi_matrix$barcodes =ra_atac_fl$barcodes

In [ ]:
lsi_matrix%>%head()

In [ ]:
# save LSI matrix
lsi_matrix%>%write_tsv(file.path(output_path, 'preRA_teaseq_rbBR2024_lsi_matrix.tsv'))

In [ ]:
getAvailableMatrices(ra_atac_fl)

In [ ]:
# get gene scores
gene_scores = getMatrixFromProject(ra_atac_fl, 'GeneScoreMatrix')

In [ ]:
# gene_scores_df = assay(gene_scores, 'GeneScoreMatrix')%>%t()%>%as.data.table()
# gene_scores_df$barcodes =ra_atac_fl$barcodes

In [ ]:
library('SingleCellExperiment')
gene_scores = as(gene_scores, "SingleCellExperiment")
gene_scores

In [ ]:
rowData(gene_scores)

In [ ]:
rowData(gene_scores)<-rowData(gene_scores)%>%as.data.frame()
colData(gene_scores)$Sample = NULL

In [ ]:
zellkonverter::writeH5AD(gene_scores, (file.path(output_path, 'preRA_teaseq_GeneScoreMatrix.h5ad')))

In [ ]:
# get tf activity scores
tf_scores = getMatrixFromProject(ra_atac_fl, 'MotifMatrix')

In [ ]:
rowData(tf_scores)<-rowData(tf_scores)%>%as.data.frame()
colData(tf_scores)$Sample = NULL

In [ ]:
tf_scores

In [ ]:
tf_scores = as(tf_scores, "SingleCellExperiment")
zellkonverter::writeH5AD(tf_scores, X_name = 'z',
                         (file.path(output_path, 'preRA_teaseq_TFscores.h5ad')))

## call peaks in MOCHA for all l2 cell types and run differential

In [ ]:
# cellColData = getCellColData(ra_atac_fl)
# cellPopLabel = 'clean_l2_cell_types'
# cellPopulations= "ALL"
# cellTypeLabelList <- cellColData[, cellPopLabel]
# cellCounts <- as.data.frame(table(cellColData[, "subject_id"], cellTypeLabelList))
# names(cellCounts) <- c("Sample", "CellPop", "CellCount")
#   cellCounts <- tidyr::pivot_wider(
#     cellCounts,
#     id_cols = "CellPop",
#     names_from = "Sample",
#     values_from = "CellCount"
#   )
# allCellCounts <- as.data.frame(cellCounts[, -1])
# rownames(allCellCounts) <- cellCounts$CellPop
# # Create a dummy data.frame for storing fragment information
#   # as it is calculated later (when iterating over cell populations)
#   allFragmentCounts <- allCellCounts
#   allFragmentCounts[!is.na(allFragmentCounts)] <- NA

#   if (all(cellPopulations == "ALL")) {
#     cellPopulations <- colnames(allCellCounts)
#   } else {
#     allCellCounts <- allCellCounts[rownames(allCellCounts) %in% cellPopulations, , drop = FALSE]
#     allFragmentCounts <- allFragmentCounts[rownames(allFragmentCounts) %in% cellPopulations, , drop = FALSE]
#   }


In [ ]:
allCellCounts['non_switched_effector_b_cells', ]

In [ ]:
####################################################
# 2. Call open tiles (main peak calling step)
#    Done once for all specified cell populations
####################################################
l2_tileResults <- MOCHA::callOpenTiles(
    ra_atac_fl,  
    cellPopLabel = 'clean_l2_cell_types',
    cellPopulations = cellPopulations,
    numCores = 58, 
    outDir = '/home/jupyter/data/preRA_teaseq/EXP-00243/ArchRSubset/MOCHA',
    TxDb = 'TxDb.Hsapiens.UCSC.hg38.refGene', 
    OrgDb = 'org.Hs.eg.db'
)


In [ ]:
l2_tileResults


In [ ]:
# ####################################################
# # 2. Call open tiles (main peak calling step)
# #    Done once for all specified cell populations
# ####################################################
# l2_celltype_tileResults <- MOCHA::callOpenTiles(
#     ra_atac,  
#     cellPopLabel = 'clean_l2_cell_types',
#     cellPopulations = 'cd14mono',
#     numCores = 58, 
#    # studySignal = studySignal,
#     outDir = '/home/jupyter/data/preRA_teaseq/EXP-00243/MOCHA',
#     TxDb = 'TxDb.Hsapiens.UCSC.hg38.refGene', 
#     OrgDb = 'org.Hs.eg.db'
# )


In [ ]:
l2_tileResults

In [ ]:
# load the tile matrix from scmacs
l2_tileResults %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_atac_l2_celltype_tiles_matrix.rds')))

In [ ]:
# load the tile matrix from scmacs
l2_tileResults <- readRDS(file.path(output_path,
                                              'preRA_teaseq_MOCHA_atac_l2_celltype_tiles_matrix.rds'))

In [ ]:
# # Parameters for downstream analysis
#cellPopulation <- "cd4_naive"
threshold <- 0.2
groupColumn <- "cohort"
join <- "union"
# l2_celltype_tileResults

In [ ]:
####################################################
# 3. Get reproducible sample-peak matrix
#    Done for each cell population individually
####################################################

# Parameters for downstream analysis
# cellPopulation <- "cd4_naive"
threshold <- 0.2
groupColumn <- "cohort"
join <- "union"
numCores=60
SampleTileMatrices <- MOCHA::getSampleTileMatrix(
    l2_tileResults,
    cellPopulations = cellPopulations,
    groupColumn = groupColumn,
    threshold = threshold,
  #  log2Intensity = TRUE,
    numCores = numCores
)

In [ ]:
SampleTileMatrices

In [ ]:
####################################################
# 4. Add gene annotations to our SampleTileMatrices,
#    labelling tiles as either a promoter, exonic,
#    intronic, or distal region. Gene names are 
#    given for all but distal. This info will aid 
#    further downstream analyses but is not required 
#    for differential accessibility.
#    This function can also take any GRanges object
#    and add annotations to its metadata.
####################################################
SampleTileMatricesAnnotated <- MOCHA::annotateTiles( 
  SampleTileMatrices
)

In [ ]:
# Load a curated motif set from library(chromVARmotifs) 
# included with ArchR installation
library(chromVARmotifs)
# library(TFBSTools)
# data("human_pwms_v2")
data('human_pwms_v2')
SampleTileMatricesAnnotated <- MOCHA::addMotifSet(
  SampleTileMatricesAnnotated, 
  motifPWMs = human_pwms_v2,  
  w = 7 # width parameter for motifmatchr::matchMotifs()
)

In [ ]:
SampleTileMatricesAnnotated %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_l2_celltype_SampleTileMatrices_rmBR2024.rds')))

In [ ]:
# # load the sample tile matrix
SampleTileMatricesAnnotated <- readRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_l2_celltype_SampleTileMatrices_rmBR2024.rds')))

In [ ]:
assays(SampleTileMatricesAnnotated)%>%names()

In [ ]:
# cd4na_Matrix <- scMACS::getCellPopMatrix(SampleTileMatricesAnnotated, "cd4_naive")
# cd4na_Matrix %>% head()

In [ ]:
SampleTileMatricesAnnotated$cohort %>% unique()
rowData(SampleTileMatricesAnnotated) %>% colnames()

In [ ]:
metadata(SampleTileMatricesAnnotated)$CellCounts %>% colnames()

In [ ]:
assays(SampleTileMatricesAnnotated)[['cd14mono']] %>% dim()
assays(SampleTileMatricesAnnotated)[['cd16mono']] %>% dim()

### run DAPs

In [ ]:
cellPopulations <- metadata(SampleTileMatricesAnnotated)$CellCounts %>% colnames()

In [ ]:
colData(SampleTileMatricesAnnotated) %>% head()

In [ ]:
# check the signal distribution in all the cell types
# getCellPopMatrix(SampleTileMatricesAnnotated, 'cd14mono') %>% head()
# extract all the tile matrix for all population
tile_dt <- lapply(cellPopulations, function(x){
    tile_mx <- getCellPopMatrix(SampleTileMatricesAnnotated, x) %>% as.data.table(keep.rownames = 'tiles') %>% 
        pivot_longer(cols = -tiles,names_to = 'Sample', values_to = 'Normalized_counts') %>% mutate(cell_type=x) %>%
        as.data.table()
    meta_data <- colData(SampleTileMatricesAnnotated) %>% as_tibble() %>% dplyr::select(subject_id, cohort, Sex, Sample)
    tile_mx <- tile_mx %>% left_join(meta_data, by='Sample') %>%
        as.data.table()
}) %>% rbindlist()

In [ ]:
tile_dt %>% head()

In [ ]:
tile_dt %>% group_by(cell_type) %>% distinct(tiles, .keep_all = TRUE) %>% tally() %>% arrange(desc(n))

In [ ]:
# summarive the median counts of tiles across cohort
tile_dt_cohort <- tile_dt %>% group_by(tiles,  cell_type) %>%
    summarise(median_normalized_counts=mean(Normalized_counts)) %>% ungroup() 
tile_dt_cohort %>% head()

In [ ]:
p1 <- tile_dt_cohort %>% ggplot()+ 
    geom_histogram(aes(x=median_normalized_counts), position = 'identity', alpha=0.6)+
    facet_wrap(vars(cell_type))+
    scale_fill_manual(values=c("#69b3a2", "#404080")) 
p1
ggsave(file.path(fig_path, paste0(proj_name, '_mocha_peaks_cell_type_cohort_avg_histogram.pdf')), 
       width = 12, height = 8)

In [ ]:
# cellPopulations <- c('naive_b_cells')
cellPopulations = c('cd14mono','cd16mono','cd4_memory','cd4_naive','naive_b_cells', 'dcs','treg', 
                    'non_switched_memory_b_cells','switched_memory_b_cells',
                    'cd8_memory','cd8_naive', 'nk_cd56dim')

In [ ]:
####################################################
# 5. Get differential accessibility for specific 
#    cell populations. Here we are comparing MAIT  
#    cells between samples where our groupColumn 
#    "COVID_status" is Positive (our foreground) 
#    to Negative samples (our background).
####################################################
groupColumn <- "cohort"
differentials_l2 <- lapply(cellPopulations, function(x){
    message(paste('running for', x))
    diff <- MOCHA::getDifferentialAccessibleTiles(
        SampleTileObj = SampleTileMatricesAnnotated,
        cellPopulation = x,
        groupColumn = groupColumn,
        foreground = "pre-RA",
        background = "Healthy",
        fdrToDisplay = 0.2, signalThreshold = 14,
        numCores = 59
    ) 
    return(diff)
})


In [ ]:
names(differentials_l2) <- cellPopulations

In [ ]:
# annotate the tiles
suppressWarnings(differentials_l2 <- lapply(differentials_l2, MOCHA::annotateTiles, 
                          TxDb = TxDb, Org = Org))

In [ ]:
# save the differential results
differentials_l2 %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_l2_celltype_DAPs_RAvsHealthy.rds')))

In [ ]:
# load the differential results
differentials_l2 <- readRDS(file.path(output_path, 'preRA_teaseq_MOCHA_l2_celltype_DAPs_RAvsHealthy.rds'))

In [ ]:
# convert to table
differentials_l2_table <- lapply(differentials_l2, as_tibble) %>% rbindlist() %>%
    filter(!is.na(FDR) & !is.na(MeanDiff)) %>% mutate(direction=if_else(MeanDiff>0, 'up', 'down')) 

In [ ]:
differentials_l2_table%>%filter(FDR<0.2)%>%write_csv(file.path(output_path, 'preRA_teaseq_MOCHA_l2_celltype_DAPs_RAvsHealthy.csv'))

In [ ]:
differentials_l2_sig = read_csv(file.path(output_path, 'preRA_teaseq_MOCHA_l2_celltype_DAPs_RAvsHealthy.csv'))

In [ ]:
differentials_l2_sig%>%distinct(CellPopulation)

In [ ]:
differentials_l2_sig%>%filter(CellPopulation%in%c('naive_b_cells', 'cd4_naive'))%>%
    mutate(Foreground='ARI', Background='CON2')%>%
   write_csv(file.path(output_path, 'preRA_teaseq_MOCHA_Tnaive_bnaive_DAPs_RAvsHealthy.csv'))

In [ ]:
differentials_l2_table %>%
    dplyr::filter( P_value<0.05 & FDR <0.2 & str_detect(Gene, 'TNFSF11'))

In [ ]:
differentials_l2_table$P_value %>% hist()

In [ ]:
# plot numbers of DAPs
dap_counts <- differentials_l2_table %>%
   # mutate(cell_type=factor(cell_type, levels = l2_cellcounts$clean_l2_cell_types)) %>% 
    dplyr::filter(FDR<0.2 & P_value<0.05) %>% 
    group_by(CellPopulation, direction) %>% dplyr::summarise(dap_counts=n())  %>%
    mutate(dap_counts=if_else(direction=='up', dap_counts, -dap_counts),
          enriched=if_else(direction=='up', 'At-risk', 'Healthy')) %>% ungroup() %>%
    arrange(desc(abs(dap_counts))) 
dap_counts %>% head()

In [ ]:
dap_counts

In [ ]:
#plot the deg counts
p1 <- ggplot(dap_counts, aes(width = 0.5, fill=enriched, y=dap_counts, x=CellPopulation)) +
  geom_bar( stat="identity", color="black")  + 
  geom_hline(yintercept=0, color = 'white', size=2)+ 
  #scale_y_continuous(breaks = seq(-2000, 1200, by = 200)) +
  geom_text(aes(y = dap_counts + 50 * sign(dap_counts), label = abs(dap_counts)), vjust = 0, size=4) +
    ylab('Number of DAPs') +
    xlab('') +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, vjust = 1, size = 12), 
         strip.text = element_text(size=12))+
    scale_fill_manual(values =  c('#0072B5FF', '#BC3C29FF'))
# Adding line to differentiate -ve and +ve y axis
p1
ggsave(file.path(fig_path, paste0(proj_name, '_l2_celltype_MOCHA_DAPs_counts.pdf')), 
       width = 6, height = 4)

In [ ]:
# plot dot plot of top features

ggpubr::ggdotchart(dap_counts, x = "CellPopulation", y = "dap_counts",
           color = "enriched",                                # Color by groups
           palette = nejm_color, # Custom color palette
          sorting = "none",                       # Sort value in descending order
           rotate = TRUE,  ylab='Number of DAPs',                              # Rotate vertically
                   dot.size = 3,                                 # Large dot size
           y.text.col = TRUE, add = "segment",                           # Color y text by groups
           ggtheme = ggpubr::theme_pubr()                        # ggplot2 theme
           )+
  ggpubr::theme_cleveland() +  geom_hline(yintercept = 0 ,linetype=2) 
ggsave(file.path(fig_path, paste0(proj_name, '_l2_celltype_MOCHA_DAPs_counts_dotplot.pdf')), 
       width = 4, height = 4)

In [ ]:
# run differential in cd4 naive only
groupColumn <- "cohort"
cd4na_diff <- scMACS::getDifferentialAccessibleTiles(
        SampleTileObj = SampleTileMatricesAnnotated,
        cellPopulation = 'cd4_naive',
        groupColumn = groupColumn,
        foreground = "pre-RA",
        background = "Healthy",
        fdrToDisplay = 0.2,
         signalThreshold = 8, # change this to 8 for better signal in cd4 naive based on the histogram
        minZeroDiff = 0.2,
        numCores = numCores
    ) 

In [ ]:
cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) & (elementMetadata(cd4na_diff)[,'FDR']<0.2) &
          abs(elementMetadata(cd4na_diff)[,'Log2FC_C'])>0.1]

In [ ]:
SampleTileMatricesAnnotated_fl <- subsetByOverlaps(SampleTileMatricesAnnotated, 
                                                   cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) & 
                                                              (elementMetadata(cd4na_diff)[,'FDR']<0.2) &
          abs(elementMetadata(cd4na_diff)[,'Log2FC_C'])>0.1])

### run DAP specifically for B naive cells

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
assays(SampleTileMatricesAnnotated)%>%names()

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)

In [ ]:
# plot intensity 
p1<-plotIntensityDistribution(
  SampleTileMatricesAnnotated,
  'naive_b_cells',
  returnDF = FALSE,
  density = TRUE
)
p2<-plotIntensityDistribution(
  SampleTileMatricesAnnotated,
  'cd4_naive',
  returnDF = FALSE,
  density = TRUE
)
p1+p2

In [ ]:
####################################################
# 5. Get differential accessibility for specific 
#    cell populations. Here we are comparing MAIT  
#    cells between samples where our groupColumn 
#    "COVID_status" is Positive (our foreground) 
#    to Negative samples (our background).
####################################################
groupColumn <- "cohort"
differentials_bnaive <- MOCHA::getDifferentialAccessibleTiles(
        SampleTileObj = SampleTileMatricesAnnotated,
        cellPopulation = 'naive_b_cells',
        groupColumn = groupColumn,
        foreground = "pre-RA",
        background = "Healthy",
        fdrToDisplay = 0.2,
        signalThreshold = 14,
        numCores = 58)


In [ ]:
differentials_bnaive$P_value%>%hist()

In [ ]:
differentials_bnaive$FDR%>%sort()%>%head()

In [ ]:
differentials_bnaive = differentials_bnaive %>% annotateTiles(TxDb = TxDb, Org = Org)

In [ ]:
differentials_bnaive%>%filter(str_detect(Gene, 'TLR9'))

## run chromvar analysis

In [ ]:
library(chromVAR)
library(chromVARmotifs)
data(human_pwms_v2)
library(BSgenome.Hsapiens.UCSC.hg38)
library(SummarizedExperiment)
library(plyranges)
# library(ChAI)

In [ ]:
# # remove the BR2024 from the matrix
# SampleTileMatricesAnnotated_fl <- 
#     SampleTileMatricesAnnotated[, colData(SampleTileMatricesAnnotated)$subject_id!='BR2024']
# SampleTileMatricesAnnotated_fl

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
# summarized_meta <- metadata(SampleTileMatricesAnnotated_fl)$summarizedData
# metadata(SampleTileMatricesAnnotated_fl)$summarizedData <- summarized_meta[, colData(summarized_meta)$subject_id!='BR2024']

In [ ]:
cellPopulations = c('cd14mono','cd16mono','cd4_memory','cd4_naive','naive_b_cells', 'dcs','treg', 
                    'non_switched_memory_b_cells','switched_memory_b_cells',
                    'cd8_memory','cd8_naive', 'nk_cd56dim')

In [ ]:
# run chromvar in cd8 naive
l2_chromvar <- lapply(cellPopulations, ChAI::makeChromVAR, 
                      atacSE = SampleTileMatricesAnnotated, numCores = 60,
                      motifName = 'Motifs')

In [ ]:
names(l2_chromvar)=cellPopulations

In [ ]:
# save the differential results
l2_chromvar %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_l2_celltype_chromavar.rds')))

In [ ]:
# save the differential results
l2_chromvar <- readRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_l2_celltype_chromavar.rds')))

In [ ]:
l2_chromvar

In [ ]:
assay(l2_chromvar$naive_b_cells$Z_Score) %>% as_tibble(rownames='tf') %>% filter(str_detect(tf, 'TBX21'))

In [ ]:
l2_chromvar[['cd4_naive']]

In [ ]:
# load the F1 scores from mofa
f1_values <- read_tsv('/home/jupyter/data/preRA_teaseq/output_results/MOFA/Mofa_preRA_cd4na_07192023_factor_values.tsv',
                     show_col_types = FALSE) %>% filter(factor=='Factor1') %>%
    dplyr::rename('subject_id'='sample', 'f1_value'='value') %>% dplyr::select(subject_id, f1_value, age)
f1_values

In [ ]:
# extract the z score for all l2 cell type
l2_zscore <- lapply(names(l2_chromvar), function(x){
    z_se <- l2_chromvar[[x]]$Z_Score
    z_score <- assay(z_se) %>% 
        as_tibble(rownames = 'motif') %>% 
        pivot_longer(cols = -motif, names_to = 'Sample', values_to = 'z_score') %>%
        left_join(as_tibble(colData(z_se)), by='Sample') %>% 
        mutate(cohort=factor(cohort, levels = c('Healthy', 'pre-RA')), 
              cell_type=x)
    return(z_score)
}) %>% rbindlist() %>% left_join(f1_values, by='subject_id')
# extract the deviation score for all l2 cell type
l2_dev <- lapply(names(l2_chromvar), function(x){
    dev_se <- l2_chromvar[[x]]$Deviations
    dev <- assay(dev_se) %>% 
        as_tibble(rownames = 'motif') %>% 
        pivot_longer(cols = -motif, names_to = 'Sample', values_to = 'dev_score') %>%
        left_join(as_tibble(colData(dev_se)), by='Sample') %>% 
        mutate(cohort=factor(cohort, levels = c('Healthy', 'pre-RA')), 
              cell_type=x)
    return(dev)
}) %>% rbindlist()%>% left_join(f1_values, by='subject_id')
l2_dev %>% head()

In [ ]:
l2_dev %>% filter(motif=='BCL6')%>%
    ggplot(aes( x=cohort,y=dev_score, color=cohort)) + 
    geom_boxplot()+ geom_jitter()+
    facet_wrap(vars(cell_type))

In [ ]:
l2_dev %>% filter(is.na(dev_score))%>%
    group_by(cell_type, motif) %>%tally() %>%arrange(n)

In [ ]:
# run wilcox test for frequency preRA vs healthy 
l2_dev_wilcox_pre <- l2_dev %>% filter(motif!='ENSG00000250542')%>%
    group_by(cell_type, motif) %>% 
    rstatix::wilcox_test(dev_score ~ cohort) %>% 
    ungroup()%>%group_by(cell_type)%>%
      rstatix::adjust_pvalue(method = "BH") %>%
      rstatix::add_significance("p.adj") 
l2_dev_wilcox_pre %>% arrange(p) %>% filter(p.adj<0.05)

In [ ]:
l2_dev_wilcox_pre

In [ ]:
# run statiscial test
# run glm as-risk vs healthy 
freq_glm <- function(freq_table, formula, celltype, motif_name){
    glm_res <- broom::tidy(stats::glm(as.formula(formula), data=freq_table)) %>%
        mutate(celltype, motif_name)
    return(glm_res)
}
dev_stats_glm <- l2_dev %>% filter(!is.na(dev_score)) %>%
    filter(str_detect(cell_type, 'cd4')#&str_detect(motif, 'NFAT|STAT|JUN|FOS|BATF|BCL6|IRF|SOX1|SOX3')
          )%>%
    group_by(cell_type, motif) %>% 
    group_modify(~freq_glm(.x, formula='f1_value ~ dev_score + age', 
                           celltype=.y$cell_type, motif_name=.y$motif))

dev_stats_glm_f1 <- freq_stats_glm %>% filter(term=='f1_value')%>% ungroup()%>%
    group_by(cell_type)%>%
    rstatix::adjust_pvalue(method = "BH") %>%
    arrange(p.value.adj) 


In [ ]:
# dev_stats_glm_f1 %>% filter(p.value.adj<0.05)

In [ ]:
# l2_dev %>% filter(str_detect(cell_type, 'cd4') & !is.na(dev_score))%>%
#     group_by(cell_type, motif) %>%tally()

In [ ]:
# l2_dev %>% #filter(str_detect(motif, 'NFAT'))%>%
#     group_by(cell_type, motif) %>% 
#     rstatix::wilcox_test(data =., dev_score ~ cohort, ref.group ='Healthy',paired = FALSE)

In [ ]:
# make a heatmap to show the p values
nfat_padj <- freq_stats_wilcox_pre %>% mutate(log10_padj=(-log10(p.adj))) %>% 
    pivot_wider(id_cols = cell_type, names_from = motif, values_from = log10_padj) %>%
    as.data.frame()
rownames(nfat_padj) <- nfat_padj$cell_type
nfat_padj$cell_type=NULL
col_fun = circlize::colorRamp2(c(-log10(0.05),  2), c("white",  "red"))
png(file.path(fig_path, paste0(proj_name, 'chromvar_z_score_nfat_preRAvsHealthy_padj_heatmap.png')), units = 'in',
    width = 8, height = 6, res=300)
ComplexHeatmap::Heatmap(as.matrix(nfat_padj), col = col_fun, name='padj')
dev.off()

In [ ]:
ComplexHeatmap::Heatmap(as.matrix(nfat_padj), col = col_fun, name='padj')

In [ ]:
sig_zscore <- l2_zscore %>% filter(str_detect(motif, 'NFAT')) %>% 
    left_join(dplyr::select(freq_stats_wilcox_pre, c(motif,cell_type,p.adj)),
                            by=c('motif','cell_type')) %>%filter(p.adj<0.05)

sig_zscore %>%
    ggplot(aes( x=cohort,y=z_score, color=cohort)) + 
    geom_boxplot()+ geom_jitter()+
    facet_grid(cols=vars(motif), rows=vars(cell_type))
ggsave(file.path(fig_path, paste0(proj_name, 'chromvar_z_score_nfat_preRAvsHealthy_boxplot.png')), 
       width = 12, height = 8)

## plot region

In [ ]:
# load the sample tile matrix
SampleTileMatricesAnnotated <- readRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_l2_celltype_SampleTileMatrices_rmBR2024.rds')))

In [ ]:
# load gene annotation database
txList <- suppressWarnings(GenomicFeatures::transcriptsBy(TxDb, by = ("gene")))
names(txList) <- suppressWarnings(AnnotationDbi::mapIds(Org, names(txList), "SYMBOL", "ENTREZID"))

In [ ]:
differentials_l2_table%>%head()

In [ ]:
# check the gene to be plot
upgene <- differentials_l2_table %>%
    dplyr::filter(CellPopulation=='cd4_memory' & !is.na(Gene) &
                  FDR<0.2 & P_value<0.05& Log2FC_C>0.5)%>%
    arrange(desc(Log2FC_C))%>%distinct(Gene)%>%pull(Gene)%>% 
    str_split(", ")%>% unlist() %>% unique()

In [ ]:
upgene%>%length()

In [ ]:
differentials_l2_table%>%distinct(CellPopulation)

In [ ]:
# pathway analysis with enrichR
quiet_library(enrichR)
setEnrichrSite("Enrichr")
dbs <- listEnrichrDbs()
# %% codecell

In [ ]:
# pathway enrichment for cd4na
enrich_cd4_na <- enrichr(upgene, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))
enrich_cd4_na[["KEGG_2021_Human"]]%>% filter(Adjusted.P.value<0.05)

In [ ]:
differentials_l2_table%>%
    dplyr::filter( P_value<0.05  &tileType=='Promoter'&
                 str_detect(Gene, 'CXCR5|PDCD1|SELL|TLR'))

In [ ]:
names(txList) %>% str_subset('CD80')

In [ ]:
# get the gene names and gene ranges
gene_name <- 'TNFSF11'
gene_ranges <- txList[[gene_name]] %>% as_tibble() %>% 
    mutate(gene_range=paste0(seqnames, ':', start-2000, '-', end+1000))
gene_ranges

In [ ]:
assays(SampleTileMatricesAnnotated)%>%names()

In [ ]:
####################################################
# 5. (Optional) Plot a specific region's coverage. 
#    Here we plot coverage at a specific region and 
#    gene by infection stage.
####################################################
cell_type='TNFSF11'
countSE <- MOCHA::extractRegion(
  SampleTileObj = SampleTileMatricesAnnotated, 
  cellPopulations = cell_type,
  #region = gene_ranges$gene_range[1], 
   region = gene_ranges[1,]$gene_range, 
  groupColumn = 'status',
  numCores = 15,
  sampleSpecific = FALSE
)


In [ ]:
# add delta
countSE <- addAccessibilityShift(countSE,
    foreground = paste0(cell_type, ".ARI"),
    background = paste0(cell_type, ".Controls"))

In [ ]:
countSE

In [ ]:
p1 <- MOCHA::plotRegion(countSE = countSE, whichGene=gene_name,  showIdeogram=TRUE,
                        counts_group_colors=c('#F59F00', '#4C8CBD', '#840032'))


In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)
p1


In [ ]:
pdf(file.path(fig_path, paste0(cell_type,'_',gene_name,'_gene_regions.pdf')), width = 6, height = 6)
p1
dev.off()

### plot regeion publication

In [ ]:
# define a function to plot gene tracks
PlotPromoterRegion <- function(SampleTileObj, gene_name, cellPopulations, groupColumn, numCores=15,
                              sampleSpecific = FALSE, ...) {
    gene_ranges <- txList[[gene_name]] %>% as_tibble() %>% 
        mutate(gene_range=paste0(seqnames, ':', start-2000, '-', end+2000))
    countSE <- MOCHA::extractRegion(
      SampleTileObj = SampleTileObj, 
      cellPopulations = cellPopulations,
      region = gene_ranges$gene_range[1], 
      groupColumn = groupColumn,
      numCores = numCores,
      sampleSpecific = sampleSpecific
    )
        # add delta
    # add delta
    countSE <- addAccessibilityShift(countSE,
        foreground = paste0(cellPopulations, ".ARI"),
        background = paste0(cellPopulations, ".Controls"))
    pdf(file.path(fig_path, paste0(proj_name,'atac_annotate_',cellPopulations, '_',
                                   gene_name,'_gene_regions.pdf')), width = 5, height = 5)
    p1 <- MOCHA::plotRegion(countSE = countSE, whichGene = gene_name, ...)
    print(p1)
    dev.off()   
    return(p1)
}

In [ ]:
# reformat the status column
colData(SampleTileMatricesAnnotated)$status = factor(if_else(colData(SampleTileMatricesAnnotated)$cohort=='pre-RA',
                                                      'ARI', 'Controls'), levels=c('Controls', 'ARI'))
SampleTileMatricesAnnotated$status%>%unique()

In [ ]:
fig_path = '/home/jupyter/figures/preRA_teaseq/ATAC'
ari_colors = c( '#840032', '#F59F00', '#5AAA46')

In [ ]:
PlotPromoterRegion(SampleTileObj = SampleTileMatricesAnnotated, gene_name = 'TLR9',
                   cellPopulations='naive_b_cells',groupColumn='status',
                   counts_group_colors=ari_colors)

In [ ]:
PlotPromoterRegion(SampleTileObj = SampleTileMatricesAnnotated, gene_name = 'CD40',
                   cellPopulations='naive_b_cells',groupColumn='status', 
                   counts_group_colors=ari_colors)

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
PlotPromoterRegion(SampleTileObj = SampleTileMatricesAnnotated, gene_name = 'TNFSF11',
                   cellPopulations='treg',groupColumn='status',
                   counts_group_colors=ari_colors)

In [ ]:
PlotPromoterRegion(SampleTileObj = SampleTileMatricesAnnotated, gene_name = 'IL23A',
                   cellPopulations='cd14mono',groupColumn='status',
                   counts_group_colors=ari_colors)

In [ ]:
PlotPromoterRegion(SampleTileObj = SampleTileMatricesAnnotated, gene_name = 'TNF',
                   cellPopulations='cd16mono',groupColumn='status',
                   counts_group_colors=ari_colors)

In [ ]:
PlotPromoterRegion(SampleTileObj = SampleTileMatricesAnnotated, gene_name = 'NFATC3',
                   cellPopulations='cd4_naive',groupColumn='status',
                   counts_group_colors=ari_colors)

In [ ]:
differentials_l2_table %>%
    dplyr::filter( P_value<0.05 & FDR<0.2 & str_detect(Gene, 'NFAT'))

In [ ]:
####################################################
# 5. Get co-accessible links between input regions
#    (tiles) and their neighboring regions within
#    a window. Here we give the first ten.
#    differential tiles as our input regions.
####################################################

cellPopulation <- cellPopulations[[4]]
cellPopulation
regions <- cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) & (elementMetadata(cd4na_diff)[,'FDR']<0.2) &
          abs(elementMetadata(cd4na_diff)[,'Log2FC_C'])>0.1]

# Alternatively, define regions as a character vector 
# of region strings in the format "chr:start-end"
# regions <- c(
#   "chrY:7326500-7326999",
#   "chrY:7327000-7327499",
#   "chrY:7339500-7339999",
#   "chrY:7344500-7344999"
# )

links <- scMACS::getCoAccessibleLinks(
    SampleTileObj = SampleTileMatricesAnnotated,
    cellPopulation = cellPopulation,
    regions = regions,
    windowSize = 1 * 10^6,
    numCores = numCores,
    verbose = TRUE
)

# Optionally filter these links by their absolute 
# correlation - this output also adds the chromosome,
# start, and end site of each link to the table.
scMACS::filterCoAccessibleLinks(links, threshold = 0.5)

In [ ]:
links

In [ ]:
# save the differential results
links %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_cd4naive_coaccess_DAPs_RAvsHealthy.rds')))

In [ ]:
# add the tile information to the differentail results
tile_info <- rowData(SampleTileMatricesAnnotated) %>% as_tibble(rownames  = 'Tile') %>%
    dplyr::select(Tile, tileType, Gene)

differentials_l2 <- differentials_l2 %>% rbindlist() %>% left_join(tile_info, by='Tile')
differentials_l2 %>% head()

In [ ]:
differentials_l2 %>% filter(FDR<0.3) %>% group_by(CellPopulation) %>% tally()

In [ ]:
differentials_l2 %>% filter(!is.na(FDR)) %>%
    ggplot(aes(x=MeanDiff, y= -log10(FDR), color=CellPopulation)) + geom_scattermore()+
    facet_wrap(vars(CellPopulation), scales = 'free')

In [ ]:
ra_atac

## import the peaks cells in scMACS into Archr and remake peak matrix

In [ ]:
# Tests if a string is a in the correct format to convert to GRanges 
validRegionString <- function(regionString) {
  if (!is.character(regionString)) {
    return(FALSE)
  }

  pattern <- "([0-9]{1,2}|chr[0-9]{1,2}):[0-9]*-[0-9]*"
  matchedPattern <- str_extract(regionString, pattern)

  if (is.na(matchedPattern)) {
    return(FALSE)
  } else if (!matchedPattern == regionString) {
    return(FALSE)
  }

  splits <- str_split(regionString, "[:-]")[[1]]
  start <- splits[2]
  end <- splits[3]
  if (start > end) {
    return(FALSE)
  }

  # All conditions satisfied
  return(TRUE)
}
# Strings to GRanges
StringsToGRanges <- function(regionString) {
  if (!validRegionString(regionString)) {
    stop("Region must be a string matching format 'seqname:start-end', where start<end e.g. chr1:123000-123500")
  }

  chrom <- gsub(":.*", "", regionString)
  startSite <- gsub(".*:", "", regionString) %>%
    gsub("-.*", "", .) %>%
    as.numeric()
  endSite <- gsub(".*-", "", regionString) %>% as.numeric()


  regionGRanges <- GRanges(seqnames = chrom, ranges = IRanges(start = startSite, end = endSite), strand = "*")
  return(regionGRanges)
}

In [ ]:
l2_peaks_called <- rowData(SampleTileMatricesAnnotated) %>% as.data.table(keep.rownames = 'peaks')
l2_peaks_called %>% colnames()

In [ ]:
l2_peaks_called %>% filter_if(is.logical, all_vars(. ==FALSE))

In [ ]:
# make a grange object from the the union of the peaks called
granges_l2 <- SampleTileMatricesAnnotated %>% rownames() %>% unique() %>% 
    MOCHA::StringsToGRanges()
granges_l2

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
## Create new feature matrix for mocha call peaks
ra_atac_fl <- addFeatureMatrix(ra_atac_fl, matrixName = "MochaPeakMatrix", features = granges_l2, binarize = FALSE)

In [ ]:
# add the peak called by scMACS back to archR
ra_atac_fl <- addPeakSet(
  ArchRProj = ra_atac_fl,
  peakSet = granges_l2,
  genomeAnnotation = getGenomeAnnotation(ra_atac_fl),
  force = TRUE
)
ra_atac_fl <- addPeakMatrix(ra_atac_fl)

In [ ]:
saveArchRProject(ra_atac_fl)

In [ ]:
getAvailableMatrices(ra_atac_fl)
# getReducedDims(ra_atac)

In [ ]:
# export the chromvar motif matrix from archr
chromvar_mx <- getMatrixFromProject(
  ArchRProj = ra_atac_fl,
  useMatrix = "MotifMatrix",
  useSeqnames = NULL,
  verbose = TRUE,
  binarize = FALSE,
  threads = getArchRThreads(),
  logFile = createLogFile("getMatrixFromProject")
)


In [ ]:
chromvar_mx

In [ ]:
# save the motif matrix by cell barcodes

#### run LSI based on the atac peak matrix called by MOCHA

In [ ]:
ra_atac <- addIterativeLSI(
    ArchRProj = ra_atac,
    useMatrix = "MochaPeakMatrix", 
    name = "MochaPeakLSI", 
    iterations = 2, 
    clusterParams = list( #See Seurat::FindClusters
        resolution = c(0.5), 
        sampleCells = 10000, 
        n.start = 10
    ), 
    varFeatures = 25000, 
    dimsToUse = 1:30
)


In [ ]:
# rerun clustering and umap on Mocha peak matrix
ra_atac <- addClusters(
  input = ra_atac,
  reducedDims = "MochaPeakLSI",
  method = "Seurat",
  name = "MochaPeak_clusters_1",
  resolution = 1,
  force = TRUE
)

ra_atac <- addUMAP(ArchRProj = ra_atac, name = "MochaPeakUMAP",
                    reducedDims = "MochaPeakLSI", force = TRUE)

In [ ]:
# plot the umap form peak matrix
p1 <- plotEmbedding(ra_atac, embedding = "MochaPeakUMAP", 
                    colorBy = "cellColData", name = c('MochaPeak_clusters_1', "l1_cell_types", "clean_l2_cell_types",
                                                     "cohort",  "subject_id"))
p2 <- plotEmbedding(ra_atac, embedding = "PeakUMAP", 
                    colorBy = "cellColData", name = c('Clusters', "l1_cell_types", "clean_l2_cell_types",
                                                     "cohort",  "subject_id"))
p3 <- plotEmbedding(ra_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = c('Clusters', "l1_cell_types", "clean_l2_cell_types",
                                                     "cohort",  "subject_id"))

plotPDF(p1,p2,p3, name = paste0(proj_name, '_MochaPeak_LSI_umap_vs_umap.pdf'), ArchRProj = ra_atac,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
getCellColData(ra_atac) %>% colnames()

In [ ]:
# assays(peak_mx)$PeakMatrix

#### identidy the different peaks feasurs in archr aomng cell types and use it to make a reference object

In [ ]:
#### Create Reference Object from ArchRProject
### Right now, I only have this function working for the peak matrix, or a tile matrix.
## @type - param to say what type of data extracted. Can be tile, peak, or gene. Default is peaks. Tile matrices will not work unless you create a new one.
## @Matrix - param for which matrix o extract
## @reduction - param for which LSI to extract. 
## @projectname - param for the project name within the new Seurat object
## @features - param for custom feature set. Set this is you generated a new tile matrix
##                of specific regions of relevance, and you want to pass it through.
## @addUMAP - param. Name of UMAP embedding to add to Seurat object, so that you don't have to recalculate the UMAP. Default is NULL, in which case it will generate a new umap based on the LSI provided. 
createReferenceProject <- function(ArchR, cellTypeLabels = NULL, maxCells=500,
                                   threshold = NULL, reduction = NULL, projectName = "RefObj", addUMAP = "UMAP"){
 
        ## Pull out markers to cluster on.
    
        if(!is.null(threshold)){
              clusterMarkers <- getMarkerFeatures(ArchR, useMatrix = "TileMatrix",
                                                  groupBy = cellTypeLabels, maxCells = maxCells,
                                            binarize = TRUE)

                clustermarkersf <- getMarkers(clusterMarkers, cutOff = threshold)
             
                allFeatures = rbindlist(lapply(seq_along(clustermarkersf), function(x){
  
                                    clustermarkersf[[x]] %>% as.data.frame() %>% 
                                                dplyr::mutate(end = start + 500)
                    })) %>% makeGRangesFromDataFrame(.,keep.extra.columns= TRUE) %>%
                plyranges::group_by(idx) %>% plyranges::reduce_ranges()
            
        }else{
            
            allFeatures <- ArchR@reducedDims$IterativeLSI$LSIFeatures %>% as.data.frame() %>% 
                            mutate(end = start + 500) %>%
                            makeGRangesFromDataFrame()  
        }
      

        ## Create new feature matrix with important info on groups. 
        ArchR <- addFeatureMatrix(ArchR, matrixName = "TransferMatrix", features = allFeatures, binarize = FALSE)

        saveArchRProject(ArchR)
    
        RefSE <- SeuratFromArchR(ArchR, type = "other", Matrix = "TransferMatrix", 
                         reduction = reduction,
                         projectName =projectName, features = allFeatures,
                          addUMAP = "UMAP")

        RefSE@assays$ATAC@var.features = rownames(RefSE)
        RefSE <- RunTFIDF(RefSE)
        RefSE <- RunSVD(RefSE)
        RefSE <- RunUMAP(RefSE, reduction = "lsi", dims = 2:50, return.model = TRUE)

        rownames(RefSE@meta.data) <- Cells(RefSE)   
        
        saveRDS(RefSE, paste(getOutputDirectory(ArchR),"/ReferenceSeurat.RDS", sep = ""))
        return(RefSE)
}

In [ ]:
so_ref_atac <- createReferenceProject(ra_atac, cellTypeLabels = 'clean_l2_cell_types', 
                                   threshold = "FDR <= 0.05 & Log2FC>0.25",
                                      maxCells = 1000,
                                      reduction = 'IterativeLSI', 
                                      projectName = "RefObj", addUMAP = "UMAP")

In [ ]:
so_ref_atac@meta.data %>% colnames()

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
p1 <- DimPlot(so_ref_atac, group.by = 'clean_l2_cell_types', cols = cluster_colors_ext, 
              shuffle = TRUE, raster=FALSE,
              label = T, reduction = 'ArchR_UMAP') + NoLegend()
p2 <- DimPlot(so_ref_atac, group.by = 'clean_l2_cell_types', cols = cluster_colors_ext, 
              shuffle = TRUE, raster=FALSE,
              label = T, reduction = 'umap') + NoLegend()
p1+p2
ggsave(file.path(fig_path, paste0(proj_name, '_refso_diff_features_vs_archr_umap.pdf')), width=12, height=6)

In [ ]:
# find marker features for different group of cells
clusterMarkers <- getMarkerFeatures(ra_atac, useMatrix = "PeakMatrix", 
                                    groupBy = 'clean_l2_cell_types',
                                    maxCells = 1000,
                                    binarize = FALSE)

In [ ]:
# get the peak features to differentiate cell types
threshold = "FDR <= 0.05 & Log2FC>0.25"
clustermarkersf <- getMarkers(clusterMarkers, cutOff = threshold)
allFeatures = rbindlist(lapply(seq_along(clustermarkersf), function(x){
    clustermarkersf[[x]] %>% as.data.frame() %>% 
        dplyr::mutate(end = start + 500)
    })) %>% makeGRangesFromDataFrame(.,keep.extra.columns= TRUE) %>%
                plyranges::group_by(idx) %>% plyranges::reduce_ranges()

In [ ]:
## Create new feature matrix with important info on groups. 
ra_atac <- addFeatureMatrix(ra_atac, matrixName = "TransferPeakMatrix", features = allFeatures, binarize = FALSE)
saveArchRProject(ra_atac)

In [ ]:
# get the peak matrix of teaseq


In [ ]:
suppressPackageStartupMessages(source('/home/jupyter/github/scATAC_Supplements/scATAC_CellTypeLabeling.R'))

In [ ]:
RefSE <- SeuratFromArchR(ra_atac, type = "other", Matrix = "TransferPeakMatrix", 
                         reduction = 'IterativeLSI',
                         projectName ="ArchRToSeurat", features = allFeatures,
                          addUMAP = "UMAP")

# RefSE@assays$ATAC@var.features = rownames(RefSE)
# RefSE <- RunTFIDF(RefSE)
# RefSE <- RunSVD(RefSE)
# RefSE <- RunUMAP(RefSE, reduction = "lsi", dims = 2:50, return.model = TRUE)

# rownames(RefSE@meta.data) <- Cells(RefSE)   
        
# saveRDS(RefSE, paste(getOutputDirectory(ArchR),"/ReferenceSeurat.RDS", sep = ""))

In [ ]:
RefSE@assays$ATAC@var.features = rownames(RefSE)
RefSE <- RunTFIDF(RefSE)
RefSE <- RunSVD(RefSE)
RefSE <- RunUMAP(RefSE, reduction = "lsi", dims = 2:50, return.model = TRUE)

rownames(RefSE@meta.data) <- Cells(RefSE)   

In [ ]:
RefSE@meta.data %>% colnames()
RefSE

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
p1 <- DimPlot(RefSE, group.by = 'clean_l2_cell_types', cols = cluster_colors_ext, 
              shuffle = TRUE, raster=FALSE,
              label = T, reduction = 'ArchR_UMAP') + NoLegend()
p2 <- DimPlot(RefSE, group.by = 'clean_l2_cell_types', cols = cluster_colors_ext, 
              shuffle = TRUE, raster=FALSE,
              label = T, reduction = 'umap') + NoLegend()
p1+p2
ggsave(file.path(fig_path, paste0(proj_name, '_refso_diff_features_vs_archr_umap.pdf')), width=12, height=6)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 15)
p1 <- DimPlot(RefSE, label = T, reduction = 'umap', shuffle = TRUE, raster=TRUE, group.by = 'Clusters') + NoLegend()
p2 <- DimPlot(RefSE, group.by = 'clean_l2_cell_types', cols = cluster_colors_ext, 
              shuffle = TRUE, raster=TRUE,
              label = T, reduction = 'umap')
p4 <- DimPlot(RefSE, group.by = 'subject_id', 
               shuffle = TRUE, raster=TRUE,
              reduction = 'umap', label = T) 
p3 <- DimPlot(RefSE, group.by = 'cohort',cols = nejm_color, shuffle = TRUE, raster=TRUE,
              reduction = 'umap')
p1+p2+p3+p4
ggsave(file.path(fig_path, paste0(proj_name, '_refso_diff_features_umap.pdf')), width=12, height=8)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 15)
p1 <- DimPlot(RefSE, label = T, reduction = 'ArchR_UMAP', shuffle = TRUE, 
              raster=TRUE, group.by = 'Clusters') + NoLegend()
p2 <- DimPlot(RefSE, group.by = 'clean_l2_cell_types', cols = cluster_colors_ext, 
              shuffle = TRUE, raster=TRUE,
              label = T, reduction = 'ArchR_UMAP')
p4 <- DimPlot(RefSE, group.by = 'subject_id', 
               shuffle = TRUE, raster=TRUE,
              reduction = 'ArchR_UMAP', label = T) 
p3 <- DimPlot(RefSE, group.by = 'cohort',cols = nejm_color, shuffle = TRUE, raster=TRUE,
              reduction = 'ArchR_UMAP')
p1+p2+p3+p4
ggsave(file.path(fig_path, paste0(proj_name, '_refso_ArchR_UMAP.pdf')), width=12, height=8)

In [ ]:
RefSE %>% saveRDS(file.path(output_path, paste0(proj_name, '_l2_cell_types_atac_seurat_ref.rds')))

In [ ]:
heatmapGS <- plotMarkerHeatmap(
  seMarker = clusterMarkers, 
  cutOff = "FDR <= 0.01 & Log2FC >= 0.5", 
  #labelMarkers = markerGenes,
  transpose = TRUE
)
pdf(file.path(fig_path, paste0(proj_name, '_refso_ArchR_diff_heatmap.pdf')), width=12, height=8)
ComplexHeatmap::draw(heatmapGS, heatmap_legend_side = "bot", annotation_legend_side = "bot")
dev.off()

### motif enrichment by chromVAR by cell types

In [ ]:
library(chromVAR)
library(motifmatchr)
library(BSgenome.Hsapiens.UCSC.hg38)
library(BiocParallel)
set.seed(2017)
library(Matrix)

In [ ]:
register(MulticoreParam(58, progressbar = TRUE))

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
#Getting GC content of peaks
SampleTileMatricesAnnotated <- addGCBias(SampleTileMatricesAnnotated, 
                            genome = BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
# get mofit annotation
CisbpAnno <- chromVAR::getAnnotations(metadata(SampleTileMatricesAnnotated)$Motifs, 
                                      rowRanges = rowRanges(SampleTileMatricesAnnotated))

In [ ]:
# Filtering inputs
#find indices of samples to keep
# sampleTileMatrics_cd4na <- filterSamples(sampleTileMatrics_cd4na, min_depth = 1500, 
#                                  min_in_peaks = 0.15, shiny = FALSE)
# sampleTileMatrics_cd4na <- chromVAR::filterPeaks(sampleTileMatrics_cd4na, non_overlapping = TRUE)


In [ ]:
SampleTileMatricesAnnotated

In [ ]:
source('/home/jupyter/github/ChAI/R/makeChromVAR.R')

In [ ]:
cellPopulation <- rowData(SampleTileMatricesAnnotated) %>% colnames()
cellPopulation

In [ ]:
# run the diviation scores
l2_chromvar <- makeChromVAR(atacSE = SampleTileMatricesAnnotated, 
                            cellPopulation = 'cd8_naive',
                           motifName='Motifs',
                            numCores =60)

In [ ]:
# run the diviation scores
cd4na_dev <- chromVAR::computeDeviations(object = SampleTileMatricesAnnotated, 
                         annotations = CisbpAnno)

In [ ]:
sampleTileMatrics_cd4na_fl

In [ ]:
motifs <- getJasparMotifs()
motif_cd4na <- matchMotifs(motifs, sampleTileMatrics_cd4na_fl, 
                        genome = BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
bg <- getBackgroundPeaks(object = sampleTileMatrics_cd4na_fl)

motif_cd4na_dev <- computeDeviations(object = sampleTileMatrics_cd4na_fl, annotations = motif_cd4na,
                                    background_peaks = bg)


In [ ]:
variability <- computeVariability(motif_cd4na_dev)
plotVariability(variability, use_plotly = FALSE)

In [ ]:
row_anno <- colData(motif_cd4na_dev)[, c('subject_id','cohort')] %>% as.data.frame()
row.names(row_anno) <- row.names(colData(motif_cd4na_dev))
row_anno

In [ ]:
sample_cor <- getSampleCorrelation(motif_cd4na_dev)

library(pheatmap)
pheatmap(as.dist(sample_cor), 
         annotation_row = row_anno,
         clustering_distance_rows = as.dist(1-sample_cor), 
         clustering_distance_cols = as.dist(1-sample_cor)
        )

In [ ]:
diff_acc <- differentialDeviations(motif_cd4na_dev, "cohort")
diff_acc %>% filter(p_value<0.01)

In [ ]:
# Motif Enrichment in Differential Peaks
ra_atac <- addMotifAnnotations(ArchRProj = ra_atac, motifSet = "cisbp", name = "Motif")

In [ ]:
# identify the enriched motif
motifsUp <- peakAnnoEnrichment(
    seMarker = l2_cell_types_Peaks,
    ArchRProj = ra_atac,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
  )

In [ ]:
heatmapEM <- plotEnrichHeatmap(motifsUp, n = 5, transpose = TRUE)
ComplexHeatmap::draw(heatmapEM, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapEM, name = "l2_cell_types_Peaks_Motif-Heatmap", 
        width = 8, height = 6, ArchRProj = ra_atac, addDOC = FALSE)

In [ ]:
# add Encode TF Binding Sites
ra_atac <- addArchRAnnotations(ArchRProj = ra_atac, collection = "EncodeTFBS")

In [ ]:
enrichEncode <- peakAnnoEnrichment(
    seMarker = l2_cell_types_Peaks,
    ArchRProj = ra_atac,
    peakAnnotation = "EncodeTFBS",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
  )

In [ ]:
heatmapEncode <- plotEnrichHeatmap(enrichEncode, n = 7, transpose = TRUE)
ComplexHeatmap::draw(heatmapEncode, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapEncode, name = "EncodeTFBS-Enriched-l2_cell_types_Markers-Heatmap", width = 8, height = 6, 
        ArchRProj = ra_atac, addDOC = FALSE)

In [ ]:
ra_atac_fl

In [ ]:
# Motif Deviations
ra_atac <- addBgdPeaks(ra_atac_fl)

In [ ]:
ra_atac <- addDeviationsMatrix(
  ArchRProj = ra_atac, 
  peakAnnotation = "Motif",
  force = TRUE
)

In [ ]:
getAvailableMatrices(ra_atac_fl)

In [ ]:
saveArchRProject(ArchRProj = ra_atac_fl)

### plot TF scores in all cell types

In [ ]:
# get motif matrix 
ra_atac_chromvar <- getMatrixFromProject(ArchRProj = ra_atac_fl, useMatrix = "MotifMatrix")

In [ ]:
# ra_atac_chromvar <- as(ra_atac_chromvar, "SingleCellExperiment")
ra_atac_chromvar

In [ ]:
colData(ra_atac_chromvar)%>%as_tibble()%>%colnames()

In [ ]:
# extract the z_scores 
metadata<-colData(ra_atac_chromvar)%>%as_tibble()%>%
    dplyr::select(c(barcodes, subject_id, cohort, clean_l2_cell_types))
ra_atac_chromvar_z <- assay(ra_atac_chromvar, 'z')
colnames(ra_atac_chromvar_z) = str_split(colnames(ra_atac_chromvar_z), pattern = '#',simplify = TRUE)[, 2]
ra_atac_chromvar_z <- ra_atac_chromvar_z %>%t()%>%as_tibble(rownames = 'barcodes') %>%
    left_join(metadata, by='barcodes')# join the metadate
ra_atac_chromvar_z %>%head()

In [ ]:
colnames(ra_atac_chromvar_z)%>%str_subset('SOX4')

In [ ]:
ra_atac_chromvar_z%>%ggplot(aes(y=clean_l2_cell_types, x=BCL6_187, fill=clean_l2_cell_types))+ geom_density_ridges()


In [ ]:
# plot TF scores in all cell types
motifs <- c('RORC', 'TBX21', 'IKZF2', 'FOXP3', 'STAT3', 'STAT5', 'BCL6','PRDM1', 'JUND', 'SOX4', 'MAF')
#motifs <-plotVarDev %>% as_tibble() %>% distinct(name) %>% pull(name)
markerMotifs <- getFeatures(ra_atac_fl, select = paste(motifs, collapse="|"), useMatrix = "MotifMatrix")
markerMotifs
markerMotifs <- grep("z:", markerMotifs, value = TRUE)
markerMotifs

In [ ]:
ra_atac_fl

In [ ]:
p <- plotGroups(ArchRProj = ra_atac_fl, 
  groupBy = "clean_l2_cell_types", 
  colorBy = "MotifMatrix", 
  name = 'z:FOXP3_348',
                maxCells = 5000,
  imputeWeights = getImputeWeights(ra_atac_fl)
)
p

In [ ]:
# load the atac clusters in cd4 t cells and visulize the tf deviation 
# load cluster identity in archr and load in mudata
cd4_archr_cluster = read_csv('/home/jupyter/data/preRA_teaseq/output_results/cd4_t/atac/PreRA_teaseq_cd4_t_atac_Clusters_0.8_cluster_barcodes.csv')
cd4_archr_cluster%>%head()

In [ ]:
cd4_chromvar_z <- ra_atac_chromvar_z%>%inner_join(cd4_archr_cluster, 'barcodes')
cd4_chromvar_z%>%head()

In [ ]:
# take a long table for tfs
cd4_chromvar_zlong <- cd4_chromvar_z%>%
    pivot_longer(cols = TFAP2B_1:TBX22_870, names_to = 'tf', values_to = 'z_scores')%>%
    mutate(C2=(Archr_Clusters_0_8=='C2'))

In [ ]:
cd4_chromvar_zlong%>%head()

In [ ]:
'z:TBX21_780', 
'z:STAT5B_779', 
'z:STAT3_777', 
'z:STAT5A_774', 
'z:SOX4_754', 
'z:RORC_681', 
'z:FOXP3_348', 
'z:BCL6B_218', 'z:PRDM16_211''z:BCL6_187''z:PRDM1_163''z:MAFB_150''z:MAFK_149''z:MAFG_148''z:MAFF_147''z:MAFA_146''z:MAF_144''z:JUND_124'

In [ ]:
motif_list<-c('TBX21_780', 'STAT5B_779', 'STAT3_777', 'SOX4_754', 'RORC_681', 'FOXP3_348',
             'PRDM1_163', 'BCL6_187', 'MAF_144', 'JUND_124')

In [ ]:
cd4_chromvar_test<- cd4_chromvar_zlong%>%filter(tf%in%motif_list)%>%group_by(tf)%>%
    rstatix::anova_test(z_scores~Archr_Clusters_0_8)

In [ ]:
cd4_chromvar_test

In [ ]:
cd4_chromvar_z%>%ggplot(aes(y=Archr_Clusters_0_8, 
                            x=MAF_144, fill=Archr_Clusters_0_8))+ geom_density_ridges()


### add ATAC data into the seurat object via Signac

In [ ]:
getAvailableMatrices(ra_atac)

In [ ]:
binarizeVar = FALSE
GRangeFeatures <- getPeakSet(ra_atac)
RangeIDs <- paste(seqnames(GRangeFeatures),":",start(GRangeFeatures),"-",end(GRangeFeatures),sep="")
assayType = "ATAC"

In [ ]:
dataMatrix <- getMatrixFromProject(ra_atac, useMatrix = 'PeakMatrix', binarize = binarizeVar)


In [ ]:
rownames(dataMatrix)<- RangeIDs
counts2 <- assays(dataMatrix)[[1]]

In [ ]:
counts2 %>% dim()

In [ ]:
# add group coverage
ra_atac <- addGroupCoverages(ArchRProj = ra_atac, maxReplicates = 16,
                                 groupBy = "clean_l2_cell_types")

# load myeloid cells in pre-RA teaseq atac and do ananlysis

In [ ]:
# define work directories
proj_path <- '/home/jupyter/data/preRA_teaseq/EXP-00243'
setwd(proj_path)
# define a project name
proj_name <- 'preRA_tea_seq_myeloid_'
fig_path <- as.character('/home/jupyter/figures/preRA_teaseq/ATAC/myeloid')
if(!dir.exists(fig_path)) (dir.create(fig_path, recursive = TRUE))

In [ ]:
# load the archr project with SubtypeA monocytes in the PreRA teaseq
# load archR data
ra_mye_atac <- loadArchRProject(path = '/home/jupyter/data/preRA_teaseq/EXP-00243/myeloid_cells')

In [ ]:
# load the filetered myeloid data in
ra_so_mye <- readRDS('/home/jupyter/data/preRA_teaseq/EXP-00243/PreRA_teaseq_seurat_myeloid_cells.rds')

In [ ]:
ra_so_mye@meta.data %>% colnames()
getCellColData(ra_mye_atac)%>% colnames()

In [ ]:
# rename the barcode to match the cell rownames
ra_mye_atac$barcodes <- str_split(rownames(ra_mye_atac), '#', simplify = TRUE)[, 2]
# ra_mye_atac@cellColData %>% as_tibble(rownames = 'atac_id') %>% select(atac_id, barcodes)

In [ ]:
setequal(ra_mye_atac$barcodes,ra_so_mye@meta.data$barcodes)

In [ ]:
# # remove column from metadata
# drops <- c('predicted.MonocyteSubsets','predicted.MonocyteSubsets.score','subject_id',
#                              'cohort','total_pbmc_counts','Tiles_snn_res.0.8','wsnn_res.0.8', 'Tiles_snn_res.0.8_labels',
#                              'Tiles_snn_res.1','Tiles_snn_res.1_labels')
# cell_id <- ra_mye_atac@cellColData %>% rownames()
# ra_mye_atac@cellColData <- ra_mye_atac@cellColData[,!(names(ra_mye_atac@cellColData) %in% drops)]
# rownames(ra_mye_atac@cellColData) <- cell_id    

In [ ]:
rownames(getCellColData(ra_mye_atac) )[1:5]
ra_mye_atac$barcodes[1:5]

In [ ]:
# add metadata from seurat to archr
# ra_mye_atac <- MetaSotoArchr(archr=ra_mye_atac, so=ra_so_mye, 
#                            so.cols=c('barcodes', 'subject_id', 'cohort', 
#                                      'total_pbmc_counts','Tiles_snn_res.0.8', 'wsnn_res.0.8'), 
#                           id.col='barcodes')

ra_mye_atac <- MetaSotoArchr(archr=ra_mye_atac, so=ra_so_mye, 
                           so.cols=c('Tiles_snn_res.1', 'predicted.MonocyteSubsets','predicted.MonocyteSubsets.score','subject_id',
                                    'cohort','total_pbmc_counts','Tiles_snn_res.0.8', 'wsnn_res.0.8', 'predicted.celltype.l2',
                                     'Tiles_snn_res.1'), 
                          id.col='barcodes')

In [ ]:
# getReducedDims(ra_mye_atac)

In [ ]:
# add lsi in atac
ra_mye_atac <- addIterativeLSI(
      ArchRProj = ra_mye_atac,
      useMatrix = "TileMatrix", 
      name = "IterativeLSI", 
      iterations = 2,
      #varFeatures = 75000, # increase the viable features
      force = TRUE
    )

In [ ]:
# make umap from lsi
ra_mye_atac <- addUMAP(ArchRProj = ra_mye_atac, 
                    reducedDims = "IterativeLSI", force = TRUE)

In [ ]:
ra_mye_atac <- addClusters(
  input = ra_mye_atac,
  reducedDims = "IterativeLSI",
  method = "Seurat",
  name = "atacClusters_res.0.8",
  resolution = 0.8,
  force = TRUE
)

In [ ]:
p1 <- plotEmbedding(ra_mye_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = "predicted.celltype.l2")
p2 <- plotEmbedding(ra_mye_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = "atacClusters_res.0.8")
p3 <- plotEmbedding(ra_mye_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = "predicted.MonocyteSubsets")
p4 <- plotEmbedding(ra_mye_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = "Tiles_snn_res.1")
p1+ p2 +p3+p4

plotPDF(p1,p2,p3, p4, name = paste0(proj_name, '_myeloid_cells_atac_umap.pdf'),ArchRProj = ra_mye_atac,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
plotEmbedding(ra_mye_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = "atacClusters_res.0.8")

In [ ]:
# add a label based on the atac clusters Tiles_snn_res.1
ra_mye_atac$atacClusters_res.0.8_labels <- 
    case_when(ra_mye_atac$atacClusters_res.0.8 %in% c('C6','C7') ~ ('CD14mon Subtype B'),
             ra_mye_atac$atacClusters_res.0.8 %in% c('C5') ~ ('CD14mon Subtype A'),
             ra_mye_atac$atacClusters_res.0.8 %in% c('C3') ~ ('CD16mon Subtype B'),
             ra_mye_atac$atacClusters_res.0.8 %in% c('C4') ~ ('CD16mon Subtype A'),
             ra_mye_atac$atacClusters_res.0.8 %in% c('C2', 'C8') ~ ('DCs'),
             ra_mye_atac$atacClusters_res.0.8 %in% c('C1', 'C9', 'C10') ~ ('Other'))

In [ ]:
ra_mye_atac$atacClusters_res.0.8 %>% unique()

In [ ]:
getCellColData(ra_mye_atac) %>% as_tibble() %>% group_by(atacClusters_res.0.8_labels) %>% tally()
# ra_so_mye@meta.data %>% as_tibble() %>% group_by(Tiles_snn_res.1) %>% tally()


In [ ]:
getCellColData(ra_mye_atac) %>% as_tibble(rownames = 'atac_id') %>% 
    select(Tiles_snn_res.1, barcodes, atac_id)  %>% head(10)
getCellColData(ra_mye_atac) %>% as_tibble() %>% 
    select(Tiles_snn_res.1, barcodes)  %>% head(10)

### calling peaks in myeloid cells atac space

In [ ]:
# add group coverage
ra_mye_atac <- addGroupCoverages(ArchRProj = ra_mye_atac, maxReplicates = 16,
                                 groupBy = "atacClusters_res.0.8_labels")

In [ ]:
saveArchRProject(ArchRProj = ra_mye_atac, load = TRUE)

In [ ]:
# !!important
# define the path in conda environment to mac2 python packages for peak calling in archr
pathToMacs2 <- '/home/jupyter/libs/r_scrna/bin/macs2'
pathToMacs2

In [ ]:
# call peaks in ra myeloid cells 
ra_mye_atac <- addReproduciblePeakSet(threads = 60,
    ArchRProj = ra_mye_atac, 
    groupBy = "atacClusters_res.0.8_labels", 
    pathToMacs2 = pathToMacs2,
    force = TRUE
)

In [ ]:
ra_mye_atac <- addPeakMatrix(ra_mye_atac)

In [ ]:
getAvailableMatrices(ra_mye_atac)

In [ ]:
saveArchRProject(ArchRProj = ra_mye_atac)

### add ATAC data into the seurat object via Signac

In [ ]:
getAvailableMatrices(ra_mye_atac)

In [ ]:
#### Add ATAC data from ArchRProjects to Seurat
### Right now, I only have this function working for the peak matrix, or a tile matrix.
## @type - param to say what type of data extracted. Can be tile, peak, gene, or other (for custom feature set). Default is peaks. Tile matrices will not work unless you create a new one.
## @Matrix - param for which matrix o extract
## @reduction - param for which LSI to extract. 
## @projectname - param for the project name within the new Seurat object
## @features - param for custom feature set. Set this is you generated a new tile matrix
##                of specific regions of relevance, and you want to pass it through.
## @addUMAP - param. Name of UMAP embedding to add to Seurat object, so that you don't have to recalculate the UMAPDefault is NULL, in which case it will generate a new umap based on the LSI provided. 
addATACtoSeurat <- function(ArchR, So, type = "Peaks", Matrix = "PeakMatrix", reduction = NULL, projectName = "ArchRToSeurat", features = NULL, addUMAP = NULL){
    
    #Set up transfer settings for various types of matrices.
    if(grepl("peak",tolower(type))){
        
        binarizeVar = FALSE
        GRangeFeatures <- getPeakSet(ArchR)
        RangeIDs <- paste(seqnames(GRangeFeatures),":",start(GRangeFeatures),"-",end(GRangeFeatures),sep="")
        assayType = "ATAC"
        
    }else if(grepl("tile",tolower(type))){
        
        binarizeVar = TRUE
        GRangeFeatures <- getPeakSet(ArchR)
        assayType = "ATAC"
        
    }else if(grepl("gene|rna",tolower(type))){
        
        binarizeVar = FALSE
        RangeIDs <- getFeatures(ArchRProj = ArchR, useMatrix = Matrix)
        assayType = "RNA"
        
    }else if(!is.null(features)){
        
        binarizeVar = FALSE
        if(class(features)[[1]] == "GRanges"){
            RangeIDs <- paste(seqnames(features),":",start(features),"-",end(features),sep="")
        }else if(is.character(features)){
            RangeIDs <- features
        }else{
            stop("Feature variable is neither a GRanges, nor a character list.")
        }
       
        assayType = "ATAC"
    }else{
        stop("Error in conversion. Please check matrix name. If it isn't a gene, tile, or peak matrix, then you need to provide a feature set as  GRanges or character list")
    }

    dataMatrix <- getMatrixFromProject(ArchR, useMatrix = Matrix, binarize = binarizeVar)

    rownames(dataMatrix)<- RangeIDs
    
    counts2 <- assays(dataMatrix)[[1]]
    
    if(dim(counts2)[1] != length(RangeIDs)){
        stop("Error: Matrix is not the same length as feature set.")
    }
    
    SeuratObj <- CreateSeuratObject(counts2, project ="Query", assay= assayType)

    if(is.null(reduction)){
            SeuratObj <- RunTFIDF(SeuratObj)
            SeuratObj <- FindTopFeatures(SeuratObj, min.cutoff = 'q99')
            SeuratObj <- RunSVD(SeuratObj)
            maxDim = 2:50
    }else{
        LSI <- getReducedDims(ArchR, reducedDims = reduction)
        SeuratObj[["IterativeLSI"]] <- CreateDimReducObject(embeddings = LSI, key = "lsi_",  assay = DefaultAssay(SeuratObj)) 
        maxDim = 1:30
    }

    if(is.null(addUMAP)){
        
        SeuratObj <- RunUMAP(SeuratObj, reduction = "LSI", dims = maxDim, return.model = TRUE)
        
    }else if(is.character(addUMAP)){
        
            umap1 <- getEmbedding(ArchRProj = ArchR, embedding = addUMAP)
            SeuratObj[["ArchR_UMAP"]] <- CreateDimReducObject(embeddings = as.matrix(umap1), key = "UMAP_", 
                                                    assay = DefaultAssay(SeuratObj))
        
    }else{
        print('Warning: addUMAP is not a string. No UMAP generated.')
    }
    
    ArchRdf <- getCellColData(ArchR)  %>% as.data.frame() %>%
            dplyr::mutate(Cells = rownames(.))
    SeurMeta <- SeuratObj@meta.data %>% dplyr::mutate(Cells = rownames(.)) %>% 
            dplyr::inner_join(ArchRdf, by = "Cells")

    SeuratObj@meta.data = SeurMeta
    
    return(SeuratObj)
    
}

In [ ]:
GRangeFeatures <- getPeakSet(ArchR)
GRangeFeatures

In [ ]:
ArchR <- ra_mye_atac

binarizeVar = TRUE
GRangeFeatures <- getPeakSet(ArchR)
RangeIDs <- paste(seqnames(GRangeFeatures),":",start(GRangeFeatures),"-",end(GRangeFeatures),sep="")
assayType = "ATAC"
Matrix = "PeakMatrix"

dataMatrix <- getMatrixFromProject(ArchR, useMatrix = Matrix, binarize = binarizeVar)
rownames(dataMatrix)<- RangeIDs
counts2 <- assays(dataMatrix)[[1]]
colnames(counts2) <- str_split(colnames(counts2), '#', simplify = TRUE)[, 2]

In [ ]:
RangeIDs <- getFeatures(ArchRProj = ArchR, useMatrix = Matrix)
RangeIDs[1:5]

In [ ]:
fragment_data <- getFragmentsFromProject(ArchR)

In [ ]:
# get gene annotations for hg38
library(EnsDb.Hsapiens.v86)
library(BSgenome.Hsapiens.UCSC.hg38)

annotation <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevelsStyle(annotation) <- "UCSC"

In [ ]:
ra_so_mye[["ATAC"]] <- CreateChromatinAssay(
  counts = counts2,
  sep = c(":", "-"),
  fragments = fragment_data,
  annotation = annotation
)

In [ ]:
ra_so_mye


In [ ]:
# DefaultAssay(ra_so_mye) <- 'ATAC'
# ra_so_mye <- RunTFIDF(ra_so_mye)
# ra_so_mye <- FindTopFeatures(ra_so_mye, min.cutoff = 'q99')
# ra_so_mye <- RunSVD(ra_so_mye)

In [ ]:
# LSI <- getReducedDims(ra_mye_atac, reducedDims = 'LSI')
lsi_data <- getReducedDims(ra_mye_atac, reducedDims = 'IterativeLSI')
# rename the lsi data to seurat style barcodes
rownames(lsi_data) <- str_split(rownames(lsi_data), '#', simplify = TRUE)[, 2]
ra_so_mye[["IterativeLSI"]] <- CreateDimReducObject(embeddings = lsi_data, key = "lsit_", assay = "ATAC")

In [ ]:
ra_so_mye <- RunUMAP(ra_so_mye, reduction = 'IterativeLSI', dims = 1:30, reduction.name = 'umap.atac')

In [ ]:
ra_so_mye@meta.data %>% colnames()

In [ ]:
# 
p1 <- DimPlot(ra_so_mye, label = T, reduction = 'umap.atac', group.by = 'Tiles_snn_res.0.8') + NoLegend()
p2 <- DimPlot(ra_so_mye, group.by = 'predicted.celltype.l2', reduction = 'umap.atac', label = T) 
p3 <- DimPlot(ra_so_mye, group.by = 'predicted.MonocyteSubsets', reduction = 'umap.atac', label = T)
p4 <- DimPlot(ra_so_mye, group.by = 'wsnn_res.0.8', reduction = 'umap.atac', label = T) 

p1 + p2 + p3 + p4
ggsave(file.path(fig_path, paste0(proj_name, '_Tiles_snn_res.0.8_predictedsubA_umap.atac.pdf')), 
       width=12, height=8)

### prepare the reference seurat object for label transfer

In [ ]:
# run pca by wknn 3wnn space
DefaultAssay(ra_so_mye) <- 'ATAC'
ra_so_mye <-  ra_so_mye %>% ScaleData()
ra_so_mye <- RunSPCA(ra_so_mye, assay = 'ATAC', graph = 'wknn')

In [ ]:
# find neighbor in 3wnn space
ra_so_mye <- FindNeighbors(
  object = ra_so_mye,
    assay = 'ATAC',
  reduction = "spca",
  dims = 1:50,
  graph.name = "spca.annoy.neighbors", 
  k.param = 50,
  cache.index = TRUE,
  return.neighbor = TRUE,
  l2.norm = TRUE
)

In [ ]:
rownames(ra_so_mye[['ATAC']])[1:5]

In [ ]:
# load the filetered myeloid data in bm
bm_so_mye <- readRDS('/home/jupyter/data/BM_teaseq/BM_teaseq_seurat_Myeloid_lineage_cells.rds')

In [ ]:
# load the ArchR project
bm_mye_atac <- loadArchRProject(path = "/home/jupyter/data/BM_teaseq/myeloid_atac")

In [ ]:
ArchR <- bm_mye_atac

binarizeVar = FALSE
GRangeFeatures <- getPeakSet(ArchR)
RangeIDs <- paste(seqnames(GRangeFeatures),":",start(GRangeFeatures),"-",end(GRangeFeatures),sep="")
assayType = "ATAC"
Matrix = "PeakMatrix"

dataMatrix <- getMatrixFromProject(ArchR, useMatrix = Matrix, binarize = binarizeVar)
rownames(dataMatrix)<- RangeIDs
counts <- assays(dataMatrix)[[1]]

In [ ]:
rownames(counts) %>% length()
rownames(ra_so_mye[['ATAC']]) %>% length()

In [ ]:
table(rownames(counts) %in% rownames(ra_so_mye[['ATAC']]))

In [ ]:
# find ancohors by spca
anchors <- FindTransferAnchors(
    reference = ra_so_mye,
    query = bm_so_mye,
    k.filter = NA,
    reference.reduction = "spca", 
    reference.neighbors = "spca.annoy.neighbors", 
    dims = 1:50
  )

In [ ]:
#### Create Reference Object from ArchRProject
### Right now, I only have this function working for the peak matrix, or a tile matrix.
## @type - param to say what type of data extracted. Can be tile, peak, or gene. Default is peaks. Tile matrices will not work unless you create a new one.
## @Matrix - param for which matrix o extract
## @reduction - param for which LSI to extract. 
## @projectname - param for the project name within the new Seurat object
## @features - param for custom feature set. Set this is you generated a new tile matrix
##                of specific regions of relevance, and you want to pass it through.
## @addUMAP - param. Name of UMAP embedding to add to Seurat object, so that you don't have to recalculate the UMAP. Default is NULL, in which case it will generate a new umap based on the LSI provided. 
createReferenceProject <- function(ArchR, cellTypeLabels = NULL, 
                                   threshold = NULL, reduction = NULL, projectName = "RefObj", addUMAP = "UMAP"){
 
        ## Pull out markers to cluster on.
    
        if(!is.null(threshold)){
              clusterMarkers <- getMarkerFeatures(ArchR, useMatrix = "TileMatrix", groupBy = cellTypeLabels, 
                                            binarize = TRUE)

                clustermarkersf <- getMarkers(clusterMarkers, cutOff = threshold)
             
                allFeatures = rbindlist(lapply(seq_along(clustermarkersf), function(x){
  
                                    clustermarkersf[[x]] %>% as.data.frame() %>% 
                                                dplyr::mutate(end = start + 500)
                    })) %>% makeGRangesFromDataFrame(.,keep.extra.columns= TRUE) %>%
                plyranges::group_by(idx) %>% plyranges::reduce_ranges()
            
        }else{
            
            allFeatures <- ArchR@reducedDims$IterativeLSI$LSIFeatures %>% as.data.frame() %>% 
                            mutate(end = start + 500) %>%
                            makeGRangesFromDataFrame()  
        }
      

        ## Create new feature matrix with important info on groups. 
        ArchR <- addFeatureMatrix(ArchR, matrixName = "TransferMatrix", features = allFeatures, binarize = FALSE)

        saveArchRProject(ArchR)
    
        RefSE <- SeuratFromArchR(ArchR, type = "other", Matrix = "TransferMatrix", 
                         reduction = reduction,
                         projectName =projectName, features = allFeatures,
                          addUMAP = "UMAP")

        RefSE@assays$ATAC@var.features = rownames(RefSE)
        RefSE <- RunTFIDF(RefSE)
        RefSE <- RunSVD(RefSE)
        RefSE <- RunUMAP(RefSE, reduction = "lsi", dims = 2:50, return.model = TRUE)

        rownames(RefSE@meta.data) <- Cells(RefSE)   
        
        saveRDS(RefSE, paste(getOutputDirectory(ArchR),"/ReferenceSeurat.RDS", sep = ""))
        return(RefSE)
}

In [ ]:
# getCellColData(ra_mye_atac) %>% as_tibble() %>% group_by() %>% tally()
getCellColData(ra_mye_atac) %>% as_tibble() %>% group_by(atacClusters_res.0.8_labels) %>% tally()

In [ ]:
# find marker features for different group of cells
clusterMarkers <- getMarkerFeatures(ra_mye_atac, useMatrix = "TileMatrix", 
                                    groupBy = 'atacClusters_res.0.8_labels', 
                                    binarize = TRUE)

In [ ]:
threshold = "FDR <= 0.2 & Log2FC>0.25"
clustermarkersf <- getMarkers(clusterMarkers, cutOff = threshold)

### label transfer from atac based on MP's functions

In [ ]:
# make a seurat object
ra_mye_atac.so <- createReferenceProject(ra_mye_atac,# threshold = "FDR <= 0.2 & Log2FC>0.25",
        cellTypeLabels = 'atacClusters_res.0.8_labels')

In [ ]:
getCellColData(ra_mye_atac) %>% as_tibble() %>%
    group_by(predicted.MonocyteSubsets) %>% tally()

In [ ]:
ra_mye_atac.so@meta.data %>% colnames()
fig_path

In [ ]:
# 
p1 <- DimPlot(ra_mye_atac.so, label = T, reduction = 'umap', group.by = 'Tiles_snn_res.0.8') + NoLegend()
p2 <- DimPlot(ra_mye_atac.so, group.by = 'predicted.celltype.l2', reduction = 'umap', label = T) 
p3 <- DimPlot(ra_mye_atac.so, group.by = 'predicted.MonocyteSubsets', reduction = 'umap')
p4 <- DimPlot(ra_mye_atac.so, group.by = 'atacClusters_res.0.8_labels', reduction = 'umap', label = T) 

p1 + p2 + p3 + p4
ggsave(file.path(fig_path, paste0(proj_name, '_Tiles_snn_res.0.8_subA_labels_allfeature_umap.pdf')), 
       width=12, height=8)

In [ ]:
# 
p1 <- DimPlot(ra_mye_atac.so, label = T, reduction = 'ArchR_UMAP', group.by = 'Tiles_snn_res.0.8') + NoLegend()
p2 <- DimPlot(ra_mye_atac.so, group.by = 'predicted.celltype.l2', reduction = 'ArchR_UMAP', label = T) 
p3 <- DimPlot(ra_mye_atac.so, group.by = 'predicted.MonocyteSubsets', reduction = 'ArchR_UMAP')
p4 <- DimPlot(ra_mye_atac.so, group.by = 'atacClusters_res.0.8_labels', reduction = 'ArchR_UMAP', label = T) 

p1 + p2 + p3 + p4
ggsave(file.path(fig_path, paste0(proj_name, '_Tiles_snn_res.0.8_predictedsubA_labelsArchR_UMAP.pdf')), 
       width=12, height=8)

In [ ]:
## Get labels within LSI space using Seurat's FindTransferAnchors and MapQuery steps
## @SeurRef - param: Seurat Reference object, with an lsi
## @SeurQuery - param: Seurat Query object, with an lsi
## @labelsToTransfer - param: list of metadata labels to transfer to query.
getLabels <- function(SeurRef, SeurQuery, labelsToTransfer, referenceReduction = "lsi", ndims = 2:30){

    transfer.anchors <- FindTransferAnchors(
          reference = SeurRef,
          query = SeurQuery,
          reference.reduction = referenceReduction,
          reduction = "lsiproject",
          dims = ndims
        )
    
    SeurQuery <- MapQuery(
          anchorset = transfer.anchors,
          reference = SeurRef,
          query = SeurQuery,
          refdata = labelsToTransfer,
          reference.reduction = referenceReduction,
          new.reduction.name = "ref.lsi",
          reduction.model = 'umap'
        )
    
    return(SeurQuery)
}

In [ ]:
getCellColData(ra_mye_atac) %>% colnames()

In [ ]:
# getAvailableMatrices(ra_mye_atac)
getPeakSet(ra_mye_atac)

In [ ]:
projHeme5 <- addPeakMatrix(projHeme4)

## load CD14 mono only in pre-RA teaseq atac and do ananlysis

In [ ]:
# load ATAC 
# load the ArchR project for cd14 monocytes
cd14mon_atac <- loadArchRProject(path = '/home/jupyter/data/preRA_teaseq/EXP-00243/cd14mono/atac')
cd14mon_atac

In [ ]:
# load the cd14 monocytes seurat object
# load the seurat obeject 
cd14_so_mye <- readRDS('PreRA_teaseq_seurat_cd14_monocytes.rds')


In [ ]:
cd14_so_mye@meta.data %>% colnames()
cd14_so_mye

In [ ]:
# test imputed gene scores of certain genes
markerGenes <- c('PLCG2')

p <- plotEmbedding(
    ArchRProj = cd14mon_atac, 
    colorBy = "GeneScoreMatrix", 
    name = markerGenes, 
    embedding = "UMAP",
    quantCut = c(0.01, 0.95),
    imputeWeights = NULL
)
p

In [ ]:
# add metadata to archr object
getCellColData(cd14mon_atac) %>% colnames()
cd14_so_mye@meta.data %>%colnames()
# extract the columns from seurat to add to archr
seurat_metadata <- cd14_so_mye@meta.data %>% 
    select(c(barcodes, subject_id, c1_monocyte_class)) %>% as_tibble()
# all(CD8_temra_atac$barcodes %in% seurat_metadata$barcodes)

#atac_metadata <- getCellColData(CD8_temra_atac) %>% as_tibble() %>% left_join(seurat_metadata, by = 'barcodes')
# add metadata from seurat to archR

cell_id <- cd14mon_atac@cellColData %>% rownames()
cd14mon_atac@cellColData <- cd14mon_atac@cellColData %>% merge(seurat_metadata, by = 'barcodes')
rownames(cd14mon_atac@cellColData ) <- cell_id

In [ ]:
# add group coverage
mye_atac <- addGroupCoverages(ArchRProj = mye_atac, groupBy = "Clusters")

# load CD4 t cells in pre-RA teaseq atac and do ananlysis

In [ ]:
# define work directories
proj_path <- '/home/jupyter/data/preRA_teaseq/EXP-00243/cd4t_cells'
setwd(proj_path)
# define a project name
proj_name <- 'preRA_tea_seq_cd4t_cells'
fig_path <- as.character('/home/jupyter/figures/preRA_teaseq/ATAC/cd4t_cells')
if(!dir.exists(fig_path)) (dir.create(fig_path, recursive = TRUE))

In [ ]:
# load ATAC 
# load the ArchR project for CD4 t cells
cd4t_atac <- loadArchRProject(path = '/home/jupyter/data/preRA_teaseq/EXP-00243/cd4t_cells')

In [ ]:
plotEmbedding(cd4t_mem_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = c("l2_cell_types", "Clusters", 'cohort'))

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4t_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = c("l2_cell_types", "Clusters"))
p2 <- plotEmbedding(cd4t_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = "l2_cell_types")
p3 <- plotEmbedding(
    ArchRProj = cd4t_atac, 
    colorBy = "GeneScoreMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4t_atac)
)
p1+ p2 + p3

plotPDF(p1,p2,p3, name = paste0(proj_name, '_atac_umap_gene_score.pdf'), ArchRProj = cd4t_atac,
        addDOC = FALSE, width = 6, height = 6)

## load cd4 mem t cells object and see if atac cluster out th types

In [ ]:
# define work directories
proj_path <- '/home/jupyter/data/preRA_teaseq/EXP-00243/cd4tmem_cells'
setwd(proj_path)
# define a project name
proj_name <- 'preRA_tea_seq_cd4t_mem'
fig_path <- as.character('/home/jupyter/figures/preRA_teaseq/ATAC/cd4t_mem')
if(!dir.exists(fig_path)) (dir.create(fig_path, recursive = TRUE))


In [ ]:
# load ATAC 
# load the ArchR project for CD4 t cells
cd4t_mem_atac <- loadArchRProject(path = '/home/jupyter/data/preRA_teaseq/EXP-00243/cd4tmem_cells')

In [ ]:
# load the seurat object
ra_tea_cd4mem <- readRDS(file.path(data_path, 'PreRA_teaseq_seurat_cd4mem_clean_t_cells.rds'))

In [ ]:
ra_tea_cd4mem@meta.data %>% colnames()
getCellColData(cd4t_mem_atac) %>% colnames()

In [ ]:
# add metadata from seurat to archr
cd4t_mem_atac <- MetaSotoArchr(archr=cd4t_mem_atac, so=ra_tea_cd4mem, 
                           so.cols=c('barcodes', 'predicted.celltype_l2', 'wsnn_res.0.5'), 
                          id.col='barcodes')

In [ ]:
# add metadata from seurat to archr
cd4t_mem_atac <- MetaSotoArchr(archr=cd4t_mem_atac, so=ra_tea_cd4mem, 
                           so.cols=c('barcodes', 't_mem_anno'), 
                          id.col='barcodes')

In [ ]:
cd4t_mem_atac <- addClusters(
      input = cd4t_mem_atac,
      reducedDims = "IterativeLSI",
      method = "Seurat",
      name = "Clusters_0.5",
      resolution = 0.5,
      force = TRUE
    )

In [ ]:
getAvailableMatrices(cd4t_mem_atac)
getCellColData(cd4t_mem_atac) %>% colnames()

### add adt umap and gene matrix into archr

In [ ]:
# cd4t_atac_gene_cores
ra_tea_cd4mem@meta.data %>% colnames() %>% sort()
all(ra_tea_cd4mem@meta.data$atac_cell_id %in% rownames(cd4t_mem_atac))
all(rownames(cd4t_mem_atac) %in% ra_tea_cd4mem@meta.data$atac_cell_id)

In [ ]:
# extracts the RNA counts from a Seurat object and adds them to an ArchR Project
cd4t_mem_atac <- addSeuratRNA(ra_tea_cd4mem, cd4t_mem_atac, 'atac_cell_id',  assay = "SCT")

In [ ]:
# # extracts the adt counts from a Seurat object and adds them to an ArchR Project
# cd4t_mem_atac <- addSeuratRNA(ra_tea_cd4mem, cd4t_mem_atac, 'atac_cell_id',  assay = "cleanadt")

In [ ]:
# add umap from seurat to addEmbedding 
adt_umap <- ra_tea_cd4mem@reductions$adt_umap@cell.embeddings %>% as.data.frame()
# rename the rownmaes to match atac id
rownames(adt_umap) <- ra_tea_cd4mem@meta.data$atac_cell_id
cd4t_mem_atac <- addEmbedding(cd4t_mem_atac, name='adt_umap',  dfEmbedding=adt_umap)

In [ ]:
saveArchRProject(ArchRProj = cd4t_mem_atac)

In [ ]:
# marker_genes <- rowData(cd4t_atac_gene_cores)$name %>% str_subset('GATA3|RORC|FOXP3|TBX21|IKZF2|STAT3')
marker_genes <- c('GATA3', 'RORC', 'FOXP3', 'TBX21', 'IKZF2', 'STAT3', 'IL17A', 'IFNG', 'IFNG')

In [ ]:
getCellColData(cd4t_mem_atac) %>% colnames()

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4t_mem_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = c('t_mem_anno',"l2_cell_types", "Clusters_0.5",
                                                      'wsnn_res.0.5',
                                                      'cohort', 'subject_id'))
p2 <-  plotEmbedding(
    ArchRProj = cd4t_mem_atac, 
    colorBy = "GeneScoreMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4t_mem_atac)
)

plotPDF(p1,p2, name = paste0(proj_name, '_Clusters_0.8_atac_umap_gene_score.pdf'), ArchRProj = cd4t_mem_atac,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4t_mem_atac, embedding = "adt_umap", 
                    colorBy = "cellColData", name = c('t_mem_anno',"l2_cell_types", "Clusters_0.5",
                                                      'wsnn_res.0.5',
                                                      'cohort', 'subject_id'))
p2 <-  plotEmbedding(
    ArchRProj = cd4t_mem_atac, 
    colorBy = "GeneScoreMatrix", 
    name = marker_genes, 
    embedding = "adt_umap",
    imputeWeights = getImputeWeights(cd4t_mem_atac)
)

plotPDF(p1,p2, name = paste0(proj_name, '_Clusters_0.8_adt_umap_gene_score.pdf'), ArchRProj = cd4t_mem_atac,
        addDOC = FALSE, width = 6, height = 6)

### call peaks in the archR (MACS2)

In [ ]:
# # add group coverage
# cd4t_mem_atac <- addGroupCoverages(ArchRProj = cd4t_mem_atac, maxReplicates = 16,
#                                  groupBy = "Clusters_0.8")

### call peaks in scMACS and run differential

In [ ]:
#Load scMACS and accompanying libraries
library(scMACS)
library(data.table)
library(ArchR)
library(GenomicRanges)
library(plyranges)

In [ ]:
getCellColData(cd4t_mem_atac) %>% as_tibble() %>% 
    group_by(t_mem_anno, subject_id) %>% tally()

In [ ]:
getCellColData(cd4t_mem_atac) %>% as_tibble() %>% 
    group_by(t_mem_anno, subject_id) %>% tally()

In [ ]:
getCellColData(cd4t_mem_atac) %>% colnames()

In [ ]:
# Define your annotation package for TxDb object(s)
# and genome-wide annotation
# Here our samples are human using hg38 as a reference.
# For more info: https://bioconductor.org/packages/3.15/data/annotation/
library(TxDb.Hsapiens.UCSC.hg38.refGene)
library(org.Hs.eg.db)
TxDb <- TxDb.Hsapiens.UCSC.hg38.refGene
Org <- org.Hs.eg.db

In [ ]:
# # set up the parameters to call peaks 
# # Parameters for calling open tiles
# cellPopulations <- cd4t_mem_atac$Clusters_0.5 %>% unique()
# cellPopLabel <- "Clusters_0.5"
# numCores=60


In [ ]:
# # call peaks in total CD4 naive cells
cellPopulations <- cd4t_mem_atac$clean_l2_cell_types %>% unique()
cellPopLabel <- "clean_l2_cell_types"
numCores=60


In [ ]:
cellPopulations

In [ ]:
####################################################
# 2. Call open tiles (main peak calling step)
#    Done once for all specified cell populations
####################################################
cd4t_mem_tileResults <- scMACS::callOpenTiles( 
    cd4t_mem_atac,
    cellPopLabel = cellPopLabel,
    cellPopulations = cellPopulations,
    TxDb = TxDb,
    Org = Org,
    numCores = numCores
)

In [ ]:
cd4t_mem_tileResults

In [ ]:
# load the tile matrix from scmacs
cd4t_mem_tileResults %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_atac_total_cd4mem_tiles_matrix.rds')))

In [ ]:
# # save thetile matrix from scmacs
# cd4na_tileResults %>% saveRDS(file.path(data_path, 
#                                   paste0(proj_name, '_scMACS_atac_total_cd4na_tiles_matrix.rds')))

In [ ]:
####################################################
# 3. Get reproducible sample-peak matrix
#    Done for each cell population individually
####################################################

# Parameters for downstream analysis
# cellPopulation <- "cd4_naive"
threshold <- 0.2
groupColumn <- "cohort"
join <- "union"

SampleTileMatrices <- scMACS::getSampleTileMatrix( 
    cd4t_mem_tileResults,
    cellPopulations = cellPopulations,
    groupColumn = groupColumn,
    threshold = threshold,
    join = join,
    NAtoZero = TRUE,
    log2Intensity = TRUE
)


In [ ]:
SampleTileMatrices %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_total_cd4mem_SampleTileMatrices.rds')))

In [ ]:
# # load the sample tile matrix
# sampleTileMatrixs <- readRDS(file.path(data_path, 
#                                   paste0(proj_name, '_scMACS_atac_Clusters_0.5_sampleTileMatrixs.rds')))

In [ ]:
SampleTileMatrices

In [ ]:
####################################################
# 4. Add gene annotations to our SampleTileMatrices,
#    labelling tiles as either a promoter, exonic,
#    intronic, or distal region. Gene names are 
#    given for all but distal. This info will aid 
#    further downstream analyses but is not required 
#    for differential accessibility.
#    This function can also take any GRanges object
#    and add annotations to its metadata.
####################################################
SampleTileMatricesAnnotated <- scMACS::annotateTiles( 
  SampleTileMatrices
)

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
####################################################
# 5. Get differential accessibility for specific 
#    cell populations. Here we are comparing MAIT  
#    cells between samples where our groupColumn 
#    "COVID_status" is Positive (our foreground) 
#    to Negative samples (our background).
####################################################

differentials <- scMACS::getDifferentialAccessibleTiles(
    SampleTileObj = SampleTileMatricesAnnotated,
    cellPopulation = "cd4_memory",
    groupColumn = groupColumn,
    foreground = "pre-RA",
    background = "Healthy",
    fdrToDisplay = 0.4,
    numCores = numCores
)

In [ ]:
# add the tile information to the differentail results
tile_info <- rowData(SampleTileMatricesAnnotated) %>% as.data.table(keep.rownames = 'Tile')
differentials <- differentials %>% left_join(tile_info, by='Tile')

In [ ]:
# save the differential results
differentials %>% fwrite(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_cd161_cd4_mem_DAPs_RAvsHealthy.tsv')))

In [ ]:
differentials %>% filter(!is.na(FDR))  %>% ggplot() + geom_histogram(aes(FDR))

In [ ]:
differentials %>% filter(FDR<0.2 & !is.na(Gene)) %>% arrange(FDR)

#### check what are the overlap of genes and atac openness

In [ ]:
# load the degs for cd4 naive
# load the deg list
ra_tea_l2_pseudo_degs <- 
    read_csv('/home/jupyter/data/preRA_teaseq/output_results/PreRA_teaseq_l2_annotation_PseudoBulk_degs_preRAvsHealthy.csv',
            show_col_types = FALSE) %>%
    filter(!is.na(FDR) & !is.na(logFC)) %>% mutate(direction=if_else(logFC>0, 'up', 'down'))
# filter cd4 naive
cd4na_pseudo_degs <- ra_tea_l2_pseudo_degs %>% filter(FDR<0.05 & cell_type=='cd4_mem')
cd4na_pseudo_degs %>% head()

In [ ]:
# filter out the differential peaks
cd4na_diff_peaks <- differentials %>% filter(FDR<0.2&!is.na(Gene)) %>% dplyr::rename('gene'='Gene')

cd4na_diff_peaks_unique <- cd4na_diff_peaks %>% arrange(FDR) %>% distinct(gene, .keep_all = TRUE)
table(cd4na_diff_peaks_unique$gene %in% cd4na_pseudo_degs$gene)

In [ ]:
cd4na_diff_peaks %>% janitor::get_dupes(gene)

In [ ]:
# combine the degs and deps
cd4na_pseudo_degs_deps <- cd4na_pseudo_degs %>% filter(gene %in% cd4na_diff_peaks_unique$gene) %>% 
    left_join(cd4na_diff_peaks_unique, suffix = c('.degs', '.deps'), by='gene')
cd4na_pseudo_degs_deps

In [ ]:
cd4na_pseudo_degs_deps %>% ggplot() + geom_point(aes(x=logFC, y=MeanDiff, color=tileType))

### import the peaks cells in scMACS into Archr and remake peak matrix

In [ ]:
# Tests if a string is a in the correct format to convert to GRanges 
validRegionString <- function(regionString) {
  if (!is.character(regionString)) {
    return(FALSE)
  }

  pattern <- "([0-9]{1,2}|chr[0-9]{1,2}):[0-9]*-[0-9]*"
  matchedPattern <- str_extract(regionString, pattern)

  if (is.na(matchedPattern)) {
    return(FALSE)
  } else if (!matchedPattern == regionString) {
    return(FALSE)
  }

  splits <- str_split(regionString, "[:-]")[[1]]
  start <- splits[2]
  end <- splits[3]
  if (start > end) {
    return(FALSE)
  }

  # All conditions satisfied
  return(TRUE)
}
# Strings to GRanges
StringsToGRanges <- function(regionString) {
  if (!validRegionString(regionString)) {
    stop("Region must be a string matching format 'seqname:start-end', where start<end e.g. chr1:123000-123500")
  }

  chrom <- gsub(":.*", "", regionString)
  startSite <- gsub(".*:", "", regionString) %>%
    gsub("-.*", "", .) %>%
    as.numeric()
  endSite <- gsub(".*-", "", regionString) %>% as.numeric()


  regionGRanges <- GRanges(seqnames = chrom, ranges = IRanges(start = startSite, end = endSite), strand = "*")
  return(regionGRanges)
}  

In [ ]:
# make a grange object from the the union of the peaks called
granges_Clusters_0.5 <- sampleTileMatrixs_data %>% pull(location) %>% unique() %>% 
    StringsToGRanges()
granges_Clusters_0.5

In [ ]:
# add the peak called by scMACS back to archR
cd4t_mem_atac <- addPeakSet(
  ArchRProj = cd4t_mem_atac,
  peakSet = granges_Clusters_0.5,
  #genomeAnnotation = getGenomeAnnotation(cd4t_mem_atac),
  force = TRUE
)


### plot regions in Archr 

In [ ]:
source('/home/jupyter/github/scATAC_Supplements/ArchR_Export_ATAC_Data.R')

In [ ]:
# Extract fragments by populations from an ArchR Project: getPopFrags()
cd4t_mem_Clusters_0.8_frags <- getPopFrags(cd4t_mem_atac, 'Clusters_0.8',cellSubsets = 'ALL' ,
                                           region = NULL, numCores = 30, 
                        NormMethod = "nfrags", blackList = NULL, overlapList = 50)

In [ ]:
cd4t_mem_Clusters_0.8_frags_files <- FragsToCoverageFiles(cd4t_mem_Clusters_0.8_frags, files = "BigWig",
                                             genome = "hg38", fname = NULL, outDir = NULL, numCores=30)

In [ ]:
getAvailableMatrices(cd4t_mem_atac)
cd4t_mem_atac <- addMotifAnnotations(ArchRProj = cd4t_mem_atac, motifSet = "JASPAR2020", name = "JasparMotifs")

In [ ]:
getAvailableMatrices(cd4t_mem_atac)

In [ ]:
cd4t_mem_c0.8_frags_RORC <- getbpCounts(regionString='chr17:47,733,236-47,746,122', 
                                          popFrags=cd4t_mem_Clusters_0.8_frags,
                                        numCores = 30, returnGRanges = FALSE)

In [ ]:
cd4t_mem_c0.8_frags_RORC %>% head()

In [ ]:
suppressMessages(p1 <- plotRegion(cd4t_mem_c0.8_frags_RORC, 
                                  ArchRProj = cd4t_mem_atac, motifSetName =  "JasparMotifs"))


In [ ]:
pdf(file.path(fig_path,paste0(proj_name, '_atac_Clusters_0.8_TBX21_gene_coverage.pdf')), width = 8, height = 8)
suppressWarnings(p1)
dev.off()

In [ ]:
# plot the heatmap of differential peaks
heatmapPeaks <- plotMarkerHeatmap(
  seMarker = cd4t_mem_atac_ravshealthy_Peaks, 
  cutOff = "FDR <= 0.1 & Log2FC >= 0.5",
  transpose = TRUE
)
draw(heatmapPeaks, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapPeaks, name = "cd4t_mem_atac_ravshealthy_Peaks_Marker-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4t_mem_atac, addDOC = FALSE)

In [ ]:
#Identifying Marker Peaks with ArchR
cd4t_mem_atac_ravshealthy_Peaks <- getMarkerFeatures(
    ArchRProj = cd4t_mem_atac, 
    useMatrix = "PeakMatrix", 
    groupBy = "cohort",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  testMethod = "wilcoxon"
)

In [ ]:
getAvailableMatrices(cd4t_atac)

## load cd4 naive t cells atac data

In [ ]:
# define work directories
proj_path <- '/home/jupyter/data/preRA_teaseq/EXP-00243/cd4tna_cells'
setwd(proj_path)
# define a project name
proj_name <- 'preRA_tea_seq_cd4tna'
fig_path <- as.character('/home/jupyter/figures/preRA_teaseq/ATAC/cd4tna')
output_path <- '/home/jupyter/data/preRA_teaseq/output_results/atac'
if(!dir.exists(fig_path)) (dir.create(fig_path, recursive = TRUE))
if(!dir.exists(output_path)) (dir.create(fig_path, recursive = TRUE))


In [ ]:
# load ATAC 
# load the ArchR project for CD4 t cells
cd4na_atac <- loadArchRProject(path = '/home/jupyter/data/preRA_teaseq/EXP-00243/cd4tna_cells')

In [ ]:
cd4na_atac

In [ ]:
# load the seurat object
ra_tea_cd4na <- readRDS(file.path(data_path, 'PreRA_teaseq_seurat_cd4naive_t_cells.rds'))

In [ ]:
ra_tea_cd4na@meta.data %>% colnames()

In [ ]:
getCellColData(cd4na_atac) %>% colnames()
cd4na_atac <- MetaSotoArchr(archr=cd4na_atac, so=ra_tea_cd4na, 
                           so.cols=c('barcodes', 'predicted.celltype_l2', 'wsnn_res.0.3','age', 'total_counts'), 
                          id.col='barcodes')

In [ ]:
getCellColData(cd4na_atac) %>% colnames()

In [ ]:
cd4na_atac <- addClusters(
      input = cd4na_atac,
      reducedDims = "IterativeLSI",
      method = "Seurat",
      name = "Clusters_0.5",
      resolution = 0.5,
      force = TRUE
    )

In [ ]:
# add arcrr clusters into seurat
atac_clusters_0.5 <- getCellColData(cd4na_atac) %>% as_tibble() %>% 
      dplyr::select(Clusters_0.5, barcodes) %>% dplyr::rename('archr_clusters_0.5'='Clusters_0.5')
cell_id <- ra_tea_cd4na@meta.data %>% rownames()
ra_tea_cd4na@meta.data <- ra_tea_cd4na@meta.data %>% left_join(atac_clusters_0.5, by = 'barcodes')
rownames(ra_tea_cd4na@meta.data) <- cell_id

In [ ]:
# load the seurat object
ra_tea_cd4na %>% saveRDS(file.path(data_path, 'PreRA_teaseq_seurat_cd4naive_t_cells.rds'))

In [ ]:
ra_tea_cd4na@meta.data %>% colnames()

In [ ]:
saveArchRProject(ArchRProj = cd4na_atac)

In [ ]:
getAvailableMatrices(cd4na_atac)
getCellColData(cd4na_atac) %>% colnames()

In [ ]:
# cd4t_atac_gene_cores <- getMatrixFromProject(cd4t_atac, useMatrix = "GeneScoreMatrix")

In [ ]:
markersGS <- getMarkerFeatures(
    ArchRProj = cd4na_atac, 
    useMatrix = "GeneScoreMatrix", 
    groupBy = "Clusters_0.5",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
markerList <- getMarkers(markersGS, cutOff = "FDR <= 0.01 & Log2FC >= 1")
markerList$C4$name

In [ ]:
# marker_genes <- rowData(cd4t_atac_gene_cores)$name %>% str_subset('GATA3|RORC|FOXP3|TBX21|IKZF2|STAT3')
marker_genes <- c('GATA3', 'RORC', 'FOXP3', 'TBX21', 'IKZF2', 'STAT3', 'SELL', 'IL17A', 'IFNG', 'IL10')
marker_genes

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4na_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = c('subject_id', 'age'))
plotPDF(p1, name = paste0(proj_name, '_subject_id_atac_umap_gene_score.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)                                                

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4na_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = c("l2_cell_types", "Clusters_0.5", 'subject_id','age',
                                                      'predicted.celltype_l2','wsnn_res.0.5',
                                                      'cohort'))
p2 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneScoreMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)

plotPDF(p1,p2, name = paste0(proj_name, '_Clusters_0.8_atac_umap_gene_score.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4na_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = c("Clusters_0.5",'cohort', 'subject_id','age'
                                                      ))
p1
plotPDF(p1, name = paste0(proj_name, '_Clusters_0.5_atac_umap.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)

### call peaks in the archR (MACS2)

In [ ]:
# add group coverage
cd4na_atac <- addGroupCoverages(ArchRProj = cd4na_atac, maxReplicates = 16,
                                 groupBy = "Clusters_0.5")

In [ ]:
# !!important
# define the path in conda environment to mac2 python packages for peak calling in archr
pathToMacs2 <- '/home/jupyter/libs/r_scrna/bin/macs2'
pathToMacs2

In [ ]:
# call peaks in cd4 naive
cd4na_atac <- addReproduciblePeakSet(threads = 60,
    ArchRProj = cd4na_atac, 
    groupBy = "Clusters_0.5", 
    pathToMacs2 = pathToMacs2,
    force = TRUE
)

In [ ]:
cd4na_atac <- addPeakMatrix(cd4na_atac)

In [ ]:
getAvailableMatrices(cd4na_atac)

In [ ]:
saveArchRProject(ArchRProj = cd4na_atac)

In [ ]:
table(cd4na_atac$Clusters_0.5)

In [ ]:
#Identifying Marker Peaks with ArchR
cd4na_clusters_0.5_Peaks <- getMarkerFeatures(
    ArchRProj = cd4na_atac, 
    useMatrix = "PeakMatrix", 
    groupBy = "Clusters_0.5",
     maxCells = 1135,
  bias = c("TSSEnrichment", "log10(nFrags)"),
  testMethod = "wilcoxon"
)

In [ ]:
markerList <- getMarkers(cd4na_clusters_0.5_Peaks, cutOff = "FDR <= 0.01 & Log2FC >= 1")
markerList

In [ ]:
# plot the heatmap of differential peaks
heatmapPeaks <- plotMarkerHeatmap(
  seMarker = cd4na_clusters_0.5_Peaks, 
  cutOff = "FDR <= 0.1 & Log2FC >= 0.5",
  transpose = TRUE
)
draw(heatmapPeaks, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapPeaks, name = "cd4na_clusters_0.5_Peaks_Marker-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4na_atac, addDOC = FALSE)

### call peaks in scMACS and run differential

In [ ]:
#Load scMACS and accompanying libraries
library(scMACS)
library(data.table)
library(ArchR)
library(GenomicRanges)
library(plyranges)

In [ ]:
getCellColData(cd4na_atac) %>% as_tibble() %>% 
    group_by(Clusters_0.5, cohort) %>% tally()

In [ ]:
getCellColData(cd4na_atac) %>% as_tibble() %>% 
    group_by(clean_l2_cell_types, cohort) %>% tally()

In [ ]:
getCellColData(cd4na_atac) %>% colnames()

In [ ]:
# Define your annotation package for TxDb object(s)
# and genome-wide annotation
# Here our samples are human using hg38 as a reference.
# For more info: https://bioconductor.org/packages/3.15/data/annotation/
library(TxDb.Hsapiens.UCSC.hg38.refGene)
library(org.Hs.eg.db)
TxDb <- TxDb.Hsapiens.UCSC.hg38.refGene
Org <- org.Hs.eg.db

In [ ]:
# # set up the parameters to call peaks 
# # Parameters for calling open tiles
# cellPopulations <- cd4na_atac$Clusters_0.5 %>% unique()
# cellPopLabel <- "Clusters_0.5"
# numCores=60


In [ ]:
# # call peaks in total CD4 naive cells
cellPopulations <- cd4na_atac$clean_l2_cell_types %>% unique()
cellPopLabel <- "clean_l2_cell_types"
numCores=60


In [ ]:
cellPopulations

In [ ]:
####################################################
# 2. Call open tiles (main peak calling step)
#    Done once for all specified cell populations
####################################################
cd4na_tileResults <- scMACS::callOpenTiles( 
    cd4na_atac,
    cellPopLabel = cellPopLabel,
    cellPopulations = cellPopulations,
    TxDb = TxDb,
    Org = Org,
    numCores = numCores
)

In [ ]:
cd4na_tileResults

In [ ]:
# load the tile matrix from scmacs
cd4na_tileResults %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_atac_total_cd4na_tiles_matrix.rds')))

In [ ]:
# # save thetile matrix from scmacs
# cd4na_tileResults %>% saveRDS(file.path(data_path, 
#                                   paste0(proj_name, '_scMACS_atac_total_cd4na_tiles_matrix.rds')))

In [ ]:
####################################################
# 3. Get reproducible sample-peak matrix
#    Done for each cell population individually
####################################################

# Parameters for downstream analysis
# cellPopulation <- "cd4_naive"
threshold <- 0.2
groupColumn <- "cohort"
join <- "union"

SampleTileMatrices <- scMACS::getSampleTileMatrix( 
    cd4na_tileResults,
    cellPopulations = cellPopulations,
    groupColumn = groupColumn,
    threshold = threshold,
    join = join,
    NAtoZero = TRUE,
    log2Intensity = TRUE
)


In [ ]:
SampleTileMatrices %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_total_cd4na_SampleTileMatrices.rds')))

In [ ]:
# # load the sample tile matrix
SampleTileMatrices <- readRDS(file.path(data_path, 
                                  paste0(proj_name, '_scMACS_atac_Clusters_0.5_sampleTileMatrixs.rds')))

In [ ]:
# # load the sample tile matrix
SampleTileMatrices <- readRDS(file.path(data_path,
                                        'preRA_tea_seq_cd4tna_scMACS_total_cd4na_SampleTileMatrices.rds'))

In [ ]:
SampleTileMatrices

In [ ]:
####################################################
# 4. Add gene annotations to our SampleTileMatrices,
#    labelling tiles as either a promoter, exonic,
#    intronic, or distal region. Gene names are 
#    given for all but distal. This info will aid 
#    further downstream analyses but is not required 
#    for differential accessibility.
#    This function can also take any GRanges object
#    and add annotations to its metadata.
####################################################
SampleTileMatricesAnnotated <- scMACS::annotateTiles( 
  SampleTileMatrices
)

In [ ]:
SampleTileMatrices$cohort %>% unique()

In [ ]:
####################################################
# 5. Get differential accessibility for specific 
#    cell populations. Here we are comparing MAIT  
#    cells between samples where our groupColumn 
#    "COVID_status" is Positive (our foreground) 
#    to Negative samples (our background).
####################################################

differentials <- scMACS::getDifferentialAccessibleTiles(
    SampleTileObj = SampleTileMatricesAnnotated,
    cellPopulation = "cd4_naive",
    groupColumn = groupColumn,
    foreground = "pre-RA",
    background = "Healthy",
    fdrToDisplay = 0.4,
    numCores = numCores
)

In [ ]:
# add the tile information to the differentail results
tile_info <- rowData(SampleTileMatricesAnnotated) %>% as.data.table(keep.rownames = 'Tile')
differentials <- differentials %>% left_join(tile_info, by='Tile')

In [ ]:
# save the differential results
differentials %>% fwrite(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_DAPs_RAvsHealthy.tsv')))

In [ ]:
# save the differential results
differentials <- fread(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_DAPs_RAvsHealthy.tsv')))

In [ ]:
differentials %>% filter(!is.na(FDR))  %>% ggplot() + geom_histogram(aes(FDR))

In [ ]:
differentials %>% filter(FDR<0.2 & !is.na(Gene))

#### check what are the overlap of genes and atac openness

In [ ]:
# load the degs for cd4 naive
# load the deg list
ra_tea_l2_pseudo_degs <- 
    read_csv('/home/jupyter/data/preRA_teaseq/output_results/PreRA_teaseq_l2_annotation_PseudoBulk_degs_preRAvsHealthy.csv',
            show_col_types = FALSE) %>%
    filter(!is.na(FDR) & !is.na(logFC)) %>% mutate(direction=if_else(logFC>0, 'up', 'down'))
# filter cd4 naive
cd4na_pseudo_degs <- ra_tea_l2_pseudo_degs %>% filter(FDR<0.05 & cell_type=='cd4_naive')
cd4na_pseudo_degs %>% head()

In [ ]:
ra_tea_l2_pseudo_degs %>% filter(gene=='PLCG2'& FDR<0.2)

In [ ]:
# filter out the differential peaks
cd4na_diff_peaks <- differentials %>% filter(FDR<0.2&!is.na(Gene)) %>% dplyr::rename('gene'='Gene')

cd4na_diff_peaks_unique <- cd4na_diff_peaks %>% arrange(FDR) %>% distinct(gene, .keep_all = TRUE)
table(cd4na_diff_peaks_unique$gene %in% cd4na_pseudo_degs$gene)

In [ ]:
cd4na_diff_peaks %>% janitor::get_dupes(gene)

In [ ]:
# combine the degs and deps
cd4na_pseudo_degs_deps <- cd4na_pseudo_degs %>% filter(gene %in% cd4na_diff_peaks_unique$gene) %>% 
    left_join(cd4na_diff_peaks_unique, suffix = c('.degs', '.deps'), by='gene')
cd4na_pseudo_degs_deps

In [ ]:
cd4na_pseudo_degs_deps %>% ggplot() + geom_point(aes(x=logFC, y=MeanDiff, color=tileType))

#### run ChromVar Motif enrichment

In [ ]:
##########
### Functions for motif analysis (enrichment and chromVar)
#' @title \code{addMotifSet}
#'
#' @description \code{addMotifSet}Identify motifs within peakset
#'
#' @param SE_Object your scMACS SummarizedExperiment. Requires Genome AnnotationDbi object within the metadata added by getSampleTileMatrix
#' @param pwms a pwms object for the motif database. Either PFMatrix, PFMatrixList, PWMatrix, or PWMatrixLis'
#' @param w the width for motifmatchr
#' @param returnObj if TRUE, return the modified SE_Object with motif set added to metadata (default). If FALSE, return the motifs from motifmatchr.
#' @param motifSetName name of the motifList in the SE_object's metadata if returnObj=TRUE. Default is 'Motifs'.
#'
#' @return the modified SE_Object with motifs added to the metadata
#' @examples
#' \dontrun{
#' # load a curated motif set from library(chromVARmotifs) included with ArchR installation
#' data(human_pwms_v2)
#' SE_with_motifs <- addMotifSet(SE_Object, pwms = human_pwms_v2, returnObj = TRUE, motifSetName = "Motifs", w = 7)
#' }
#'
#' @export

addMotifSet <- function(SE_Object, pwms, w = 7, returnObj = TRUE, motifSetName = "Motifs") {
  TotalPeakSet <- rowRanges(SE_Object)
  genome <- metadata(SE_Object)$Genome
  motif_ix <- motifmatchr::matchMotifs(
    pwms = pwms,
    TotalPeakSet,
    genome = genome,
    out = "positions", w = w
  )
  names(motif_ix) <- sub("_D_.*|_I_.*", "", names(motif_ix)) %>%
    sub("_I$|_D$", "", .) %>%
    sub(".*_LINE", "", .) %>%
    sub(".*_", "", .)

  motifList <- list(motif_ix)
  names(motifList) <- motifSetName

  if (returnObj) {
    metadata(SE_Object) <- append(metadata(SE_Object), motifList)
    return(SE_Object)
  } else {
    return(motif_ix)
  }
}
## runChromVar is a wrapper for chromVAR from scMACS
## Obj could be a ragged experiment or a sumarized experiment
## motifGRangesList is a list of all motif positions in a GRangesList format
## genome is the reference genome we use


runChromVar <- function(Obj, motifs,
                       genome =  BSgenome.Hsapiens.UCSC.hg38){
    
    if(class(Obj)[1] == "RaggedSummarizedExperiment" & class(motifs)[1]=='GRangesList'){
        
        Obj1 <- RaggedExperiment::compactSummarizedExperiment(RepTiles, i = 'TotalIntensity')
        tmp <- SummarizedExperiment::assays(Obj1)
        tmp[[1]][is.na(tmp[[1]])] = 0
        names(tmp) <- "counts"
        assays(Obj1) <- tmp

	motifGRangesList = motifs
        
    }else if(class(Obj)[1] == "RangedSummarizedExperiment"){
        
        if( !names(assays(Obj)) %in% 'counts'){
        
		
            
        }else if(class(motifs)[1] == 'GRangesList'){
            
            Obj1 = Obj
	    motifGRangesList = motifs
	    
        }else if(class(motifs)[1] == 'character' & any(names(metadata(Obj)) %in% motifs)){
            
            Obj1 = Obj
	    motifGRangesList = metadata(Obj)[[motifs]]
	    
        }else{

	    stop('Error: No motifset found. Check input')
	}
        
    }else{
    
        stop('Error: Wrong Input Object. Must be either a RaggedExperiment or a RangedSummarizedExperiment')
    }     

    CisbpAnno <- chromVAR::getAnnotations(motifGRangesList, rowRanges = rowRanges(Obj1))

    Obj1 <- chromVAR::addGCBias(Obj1, genome = genome)
    
    dev <- chromVAR::computeDeviations(object = Obj1, 
                         annotations = CisbpAnno)
    
    return(dev)
}




#################      Category          Not in Category       Total
## Group1      length(Group1Cat)        length(OnlyGroup1)       m 
## Group2      length(Group2Cat)        length(OnlyGroup2)       n
# Total                  k
                 
### Enrichment test for GRanges. Test for enrichment of Category within Group1
### @Group1 - A GRanges object for one set of positions
### @Group2 - The background GRanges object, non-overlapping with Group1.
### @Category - A GRanges object of known locations, such as motifs, that you want to test for enrichment in Group1.
### @type - Default is null. You can use this to pull out or simplify the test to a metadata column within the GRanges 
###          for Group1 and Group2. For example, if you want to test for enrichment of all genes, instead of open regions. 
###          If type = null, then it will just use the number of Ranges instead of the number of unique 
###           entries in column 'type'
EnrichedRanges <- function(Group1, Group2, Category, type = NULL, returnTable = FALSE){
    
    Group1Cat <- plyranges::filter_by_overlaps(Group1, Category) 
    Group2Cat <- plyranges::filter_by_overlaps(Group2, Category)
   
    OnlyGroup1 <- plyranges::filter_by_non_overlaps(Group1, Category)
    OnlyGroup2 <- plyranges::filter_by_non_overlaps(Group2, Category)
    
    if(returnTable & is.null(type)){

        dt_table <- data.frame(Group1 = c(length(Group1Cat), length(OnlyGroup1)), 
                               Group2 = c(length(Group2Cat), length(OnlyGroup2)), 
                       row.names = c('In Category', 'Not in Category')) 
    
        return(t(dt_table))
        
    }else if(returnTable & 
             sum(c(colnames(mcols(Group1)),colnames(mcols(Group2))) %in% type) == 2 &
             length(type) == 1){
        
       dt_table <- data.frame(Group1 = c(length(unique(GenomicRanges::mcols(Group1Cat)[,type])), 
                                          length(unique(GenomicRanges::mcols(OnlyGroup1)[,type]))), 
                               Group2 = c(length(unique(GenomicRanges::mcols(Group2Cat)[,type])), 
                                         length(unique(GenomicRanges::mcols(OnlyGroup2)[,type]))), 
                       row.names = c('In Category', 'Not in Category')) 
    
        return(t(dt_table))
       
    }else if(returnTable){
        
        stop('Error: Incorrect method or column name. Please check input')      
    }
    
    if(is.null(type)){
        
       pVal <- phyper(q = length(Group1Cat), 
           m = length(Group1), 
           n = length(Group2),
           k = length(Group1Cat) + length(Group2Cat),
            lower.tail=FALSE)

       enrichment <- (length(Group1Cat)/length(Group1))/(length(Group2Cat)/length(Group2))
        
    }else if(sum(c(colnames(mcols(Group1)),colnames(mcols(Group2))) %in% type) == 2 &
             length(type) == 1){
        
       pVal <- phyper(q = length(unique(mcols(Group1Cat)[,type])), 
           m = length(unique(mcols(Group1)[,type])), 
           n = length(unique(mcols(Group2)[,type])),
           k = length(unique(mcols(Group1Cat)[,type])) + length(unique(mcols(Group2Cat)[,type])),
                     lower.tail=FALSE)
        
       enrichment <- (length(unique(mcols(Group1Cat)[,type]))/length(unique(mcols(Group1)[,type])))/
                    (length(unique(mcols(Group2Cat)[,type]))/length(unique(mcols(Group2)[,type])))
        
    }else{
        
        stop('Error: Incorrect method or column name. Please check input')
        
    }
    
    return(data.frame(p_value = pVal, enrichment = enrichment))
    
}
    
######## Test all motifs for enrichment.
    

MotifEnrichment <- function(Group1, Group2, motifPosList, type = NULL, numCores = 1){
    
    
    allEnrichmentList <- mclapply(motifPosList, function(x){
        
        tmp_df <- EnrichedRanges(Group1, Group2, Category = x, type = type)
        
    }, mc.cores = numCores)
    df_final <- do.call('rbind',  allEnrichmentList)

    df_final$adjp_val <- p.adjust(df_final$p_value)
    df_final$mlog10Padj <- -log10(df_final$adjp_val)
    
    return(df_final)

}
    
############# Pull out all the motifs associated with each gene according to a set of TSS sites

## @TSS_Sites - GRanges objects that are the list of TSS sites of interest. 
##              Must include a column 'name' which has the associated gene name
## @allPeaks - GRanges object of all peaks 
## @TSS_Links - a data.table object that record all the peak-peak links by co-accessibility
##              Must include columns named 'Peak1' and 'Peak2' which contain a string describing
##              each peak in the format 'chr1:100-2000' and must be identical to peaks listed in
##              allPeaks
## @motifPosList - a GRangesList, which each index is a GRanges of all positions 
##                  for a given motif. GRangesList must be named. 
## @numCores - number of cores to multithread over. 
    
    
Gene2Motif <- function(TSS_Sites, allPeaks, TSS_Links, motifPosList, 
                       numCores = 1, verbose = FALSE){
    
    if(verbose){ print('Generating TSS-Peak Network.')}
    
    TSS_Network <- c(TSS_Links$Peak1, TSS_Links$Peak2, 
                           GRangesToString(TSS_Sites)) %>%
                   unique() %>%
                  StringsToGRanges(.) %>% 
                plyranges::filter_by_overlaps(allPeaks, .)
    
    if(verbose){ print('Finding all motifs related to each peak within the TSS-Peak Network.')}
    ##Let's find all the motifs that overlap with each peak within the altTSS Network
    tmpOverlap <- mclapply(seq_along(motifPosList), function(x){
    
        ifelse(count_overlaps(TSS_Network, motifPosList[[x]]) > 0,
           names(motifPosList)[x], NA)
    
    }, mc.cores= numCores)
    
    
    overlap_df <- do.call('cbind', tmpOverlap)
    motifList <- mclapply(c(1:dim(overlap_df)[1]), function(x){
        
        
        ifelse(any(!is.na(overlap_df[x,])),
            list(overlap_df[x,which(!is.na(overlap_df[x,]))]),
            NA)
    
    }, mc.cores= numCores)

    if(verbose){ print('Finding all peaks related to each gene within the TSS-Peak Network.')}
    ##Find all the peaks related to each gene. 
    
    Peak2Gene <- mclapply(unique(TSS_Sites$name), function(x){
    
        geneTSS <- plyranges::filter(TSS_Sites, name == x)  %>% 
                plyranges::filter_by_overlaps(allPeaks, .) %>%
            plyranges::ungroup() %>%
            GRangesToString(.)
    
        tmp <- TSS_Links[Peak1 %in% geneTSS | Peak2 %in% geneTSS,]
    
        if(dim(tmp)[1] > 0){ 
            unique(c(tmp$Peak1, tmp$Peak2, geneTSS))
        }else{
               geneTSS
        }
    
    }, mc.cores = numCores)
    names(Peak2Gene) <- unique(TSS_Sites$name)
    
    

    if(verbose){ print('Linking Motifs to each gene within the TSS-Peak Network')}
    ## Link all the genes to motifs via Peak2Gene and the motifList
    Gene2Motif <- mclapply(Peak2Gene, function(x){
    
        #Find which indices of the AltTSS Network GRanges are linked to that Gene
        tmp <- findOverlaps(StringsToGRanges(x), TSS_Network)
        #Pull up and unlist all the motifs associated with those tiles.
        unlist(motifList[subjectHits(tmp)])
    
    }, mc.cores = numCores)
    
    return(Gene2Motif)
    
}
    
    

In [ ]:
library(chromVARmotifs)
library(TFBSTools)
data("human_pwms_v2")
SampleTileMatrices_motifs <- addMotifSet(SampleTileMatrices, pwms = human_pwms_v2, returnObj = FALSE,
                                         motifSetName = "Motifs", w = 7)

In [ ]:
SampleTileMatrices_motifs

In [ ]:
SampleTileMatrices_dev <- runChromVar(SampleTileMatricesAnnotated, motifs =  SampleTileMatrices_motifs,
                                      genome =  BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
CisbpAnno <- chromVAR::getAnnotations(SampleTileMatrices_motifs, 
                                      rowRanges = rowRanges(SampleTileMatricesAnnotated))

# SampleTileMatricesAnnotated <- chromVAR::addGCBias(SampleTileMatricesAnnotated, genome = BSgenome.Hsapiens.UCSC.hg38)
    
# dev <- chromVAR::computeDeviations(object = SampleTileMatricesAnnotated, 
#                          annotations = CisbpAnno)

In [ ]:
SampleTileMatricesAnnotated <- chromVAR::addGCBias(SampleTileMatricesAnnotated, 
                                                   genome = BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
assayNames(SampleTileMatricesAnnotated)
assay(SampleTileMatricesAnnotated, 'counts') <- assay(SampleTileMatricesAnnotated, 'cd4_naive')

In [ ]:
cd4na_dev <- chromVAR::computeDeviations(object = SampleTileMatricesAnnotated, 
                         annotations = CisbpAnno)

In [ ]:
variability <- computeVariability(cd4na_dev)
plotVariability(variability, use_plotly = FALSE)

In [ ]:
colData(cd4na_dev)[, c('subject_id', 'cohort', 'age')]
sample_cor <- getSampleCorrelation(cd4na_dev)
sample_cor

In [ ]:
sample_cor <- getSampleCorrelation(cd4na_dev)

library(pheatmap)
pheatmap(as.dist(sample_cor), 
         annotation_row = colData(cd4na_dev)[, c('subject_id', 'cohort', 'age')], 
         clustering_distance_rows = as.dist(1-sample_cor), 
         clustering_distance_cols = as.dist(1-sample_cor))

In [ ]:
tsne_results <- deviationsTsne(cd4na_dev, threshold = 1.5, perplexity = 10, 
                               shiny = FALSE)
tsne_plots <- plotDeviationsTsne(cd4na_dev, tsne_results, annotation = "TEAD3", 
                                   sample_column = "cohort", shiny = FALSE)
tsne_plots[[2]]

In [ ]:
diff_acc <- differentialDeviations(cd4na_dev, "cohort")

head(diff_acc)

### import the peaks cells in scMACS into Archr and remake peak matrix

In [ ]:
# Tests if a string is a in the correct format to convert to GRanges 
validRegionString <- function(regionString) {
  if (!is.character(regionString)) {
    return(FALSE)
  }

  pattern <- "([0-9]{1,2}|chr[0-9]{1,2}):[0-9]*-[0-9]*"
  matchedPattern <- str_extract(regionString, pattern)

  if (is.na(matchedPattern)) {
    return(FALSE)
  } else if (!matchedPattern == regionString) {
    return(FALSE)
  }

  splits <- str_split(regionString, "[:-]")[[1]]
  start <- splits[2]
  end <- splits[3]
  if (start > end) {
    return(FALSE)
  }

  # All conditions satisfied
  return(TRUE)
}
# Strings to GRanges
StringsToGRanges <- function(regionString) {
  if (!validRegionString(regionString)) {
    stop("Region must be a string matching format 'seqname:start-end', where start<end e.g. chr1:123000-123500")
  }

  chrom <- gsub(":.*", "", regionString)
  startSite <- gsub(".*:", "", regionString) %>%
    gsub("-.*", "", .) %>%
    as.numeric()
  endSite <- gsub(".*-", "", regionString) %>% as.numeric()


  regionGRanges <- GRanges(seqnames = chrom, ranges = IRanges(start = startSite, end = endSite), strand = "*")
  return(regionGRanges)
}  

In [ ]:
SampleTileMatricesAnnotated %>% rowData()

In [ ]:
# make a grange object from the the union of the peaks called
granges_Clusters_0.5 <- SampleTileMatrices %>% pull(location) %>% unique() %>% 
    StringsToGRanges()
granges_Clusters_0.5

In [ ]:
SampleTileMatrices

In [ ]:
# add the peak called by scMACS back to archR
cd4na_atac <- addPeakSet(
  ArchRProj = cd4na_atac,
  peakSet = granges_Clusters_0.5,
  #genomeAnnotation = getGenomeAnnotation(cd4na_atac),
  force = TRUE
)


In [ ]:
# add peakmatrix based on the peak set
cd4na_atac <- addPeakMatrix(cd4na_atac)
getAvailableMatrices(cd4na_atac)

In [ ]:
# rerun a lsi based on the peakmatrix
# add lsi in atac
cd4na_atac <- addIterativeLSI(
      ArchRProj = cd4na_atac,
      useMatrix = "PeakMatrix", 
      name = "IterativeLSI_peak", 
      iterations = 2,
      #varFeatures = 75000, # increase the viable features
      force = TRUE
    )

In [ ]:
# redo clustering based on the lsi from scMACS called peak matrix
cd4na_atac <- addClusters(
    input = cd4na_atac,
    reducedDims = "IterativeLSI_peak",
    method = "Seurat",
    name = "peak_Clusters_0.5",
    resolution = 0.5,
    force = TRUE
)

In [ ]:
cd4na_atac <- addUMAP(
    ArchRProj = cd4na_atac, 
    reducedDims = "IterativeLSI_peak", 
    name = "UMAP_peak", 
    nNeighbors = 30, 
    minDist = 0.5, 
    metric = "cosine",
    force = TRUE
)

In [ ]:
saveArchRProject(ArchRProj = cd4na_atac)

In [ ]:
p1 <- cd4na_atac %>% BoxPlotClusterFreq(cluster.name='peak_Clusters_0.5',
                                        totalcounts = 'total_counts',
                                          color.by='cohort', log.y = FALSE)
p1 

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4na_atac, embedding = "UMAP_peak", 
                    colorBy = "cellColData", name = c("l2_cell_types", "peak_Clusters_0.5",'peak_Clusters_0.8',
                                                      'subject_id','age','predicted.celltype_l2','wsnn_res.0.5',
                                                      'cohort'))
plotPDF(p1, name = paste0(proj_name, '_Clusters_0.8_atac_peak_umap_gene_score.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
# plot density UMAP
plotDensity(ArchRProj= cd4na_atac, embeddingName= 'UMAP_peak', identity = 'cohort', returnObj = FALSE)

In [ ]:
cd4na_atac$peak_Clusters_0.5 %>% table()

In [ ]:
# Identifying Marker Peaks with ArchR
cd4na_clusters_0.5_c4_Peaks <- getMarkerFeatures(
    ArchRProj = cd4na_atac, 
    useMatrix = "PeakMatrix", 
   # useGroups = c('C4'),
   # bgdGroups = c('C1', 'C2', 'C3','C6'),
    groupBy = "peak_Clusters_0.5",
  bias = c("TSSEnrichment", "log10(nFrags)"),
     maxCells = 2500,
  testMethod = "wilcoxon"
)

In [ ]:
markerList <- getMarkers(cd4na_clusters_0.5_c4_Peaks, cutOff = "FDR <= 0.01 & Log2FC >= 1")
markerList[[1]]

In [ ]:
# plot the heatmap of differential peaks
heatmapPeaks <- plotMarkerHeatmap(
  seMarker = cd4na_clusters_0.5_c4_Peaks, 
  cutOff = "FDR <= 0.05 & abs(Log2FC) >= 1",
    plotLog2FC = TRUE,
  transpose = TRUE
)
draw(heatmapPeaks, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapPeaks, name = "cd4na_clusters_0.5_C4_Peaks_Marker-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4na_atac, addDOC = FALSE)

In [ ]:
# Motif Enrichment in Differential Peaks

# cisbp lower quality higher coverage
# japstat higher quality lower coverage
cd4na_atac <- addMotifAnnotations(ArchRProj = cd4na_atac, motifSet = "cisbp", name = "Motif",
                                 force = TRUE)

In [ ]:
# identify the enriched motif for all clutsers
cd4na_clusters_0.5_motif <- peakAnnoEnrichment(
    seMarker = cd4na_clusters_0.5_c4_Peaks,
    ArchRProj = cd4na_atac,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
  )

In [ ]:
heatmapEM <- plotEnrichHeatmap(cd4na_clusters_0.5_motif, n = 50, transpose = TRUE)
ComplexHeatmap::draw(heatmapEM, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapEM, name = "cd4na_clusters_0.5_Peaks_Motif-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4na_atac, addDOC = FALSE)

In [ ]:
# identify the enriched motif
# exlcuding C4
cd4na_clusters_0.5_noc4_motif <- peakAnnoEnrichment(
    seMarker = cd4na_clusters_0.5_c4_Peaks[,c(1:3, 5, 6)],
    ArchRProj = cd4na_atac,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
  )

In [ ]:
cd4na_clusters_0.5_motif[,c(1:3, 5, 6)]

In [ ]:
heatmapEM <- plotEnrichHeatmap(cd4na_clusters_0.5_noc4_motif, n = 50, transpose = TRUE)
ComplexHeatmap::draw(heatmapEM, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapEM, name = "cd4na_clusters_0.5_noC4_Peaks_Motif-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4na_atac, addDOC = FALSE)

### motif enrichment by cell types

In [ ]:
# Motif Enrichment in Differential Peaks

# cisbp lower quality higher coverage
# japstat higher quality lower coverage
cd4na_atac <- addMotifAnnotations(ArchRProj = cd4na_atac, motifSet = "cisbp", name = "Motif",
                                 force = TRUE)

In [ ]:
# identify the enriched motif
motifsEnriched <- peakAnnoEnrichment(
    seMarker = cd4na_clusters_0.5_Peaks,
    ArchRProj = cd4na_atac,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & abs(Log2FC) >= 0.5"
  )

In [ ]:
motifsEnriched

In [ ]:
heatmapEM <- plotEnrichHeatmap(motifsEnriched, n = 100, transpose = TRUE)
ComplexHeatmap::draw(heatmapEM, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapEM, name = "cd4na_clusters_0.5_Peaks_Motif-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4na_atac, addDOC = FALSE)

In [ ]:
# add Encode TF Binding Sites
cd4na_atac <- addArchRAnnotations(ArchRProj = cd4na_atac, force = TRUE,
                                  collection = "EncodeTFBS")

In [ ]:
cd4na_clusters_0.5_Peaks

In [ ]:
enrichEncode <- peakAnnoEnrichment(
    seMarker = cd4na_clusters_0.5_Peaks,
    ArchRProj = cd4na_atac,
    peakAnnotation = "EncodeTFBS",
    cutOff = "FDR <= 0.1 & abs(Log2FC) >= 0.5"
  )

In [ ]:
heatmapEncode <- plotEnrichHeatmap(enrichEncode, n = 20, transpose = TRUE)
ComplexHeatmap::draw(heatmapEncode, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapEncode, name = "EncodeTFBS-Enriched-cd4na_clusters_0.5_Markers-Heatmap", width = 8, height = 6, 
        ArchRProj = cd4na_atac, addDOC = FALSE)

In [ ]:
# Motif Deviations
cd4na_atac <- addBgdPeaks(cd4na_atac)

In [ ]:
cd4na_atac <- addDeviationsMatrix(
  ArchRProj = cd4na_atac, 
  peakAnnotation = "Motif",
  force = TRUE
)

In [ ]:
plotVarDev <- getVarDeviations(cd4na_atac, name = "MotifMatrix", plot = TRUE)
plotVarDev
plotPDF(plotVarDev, name = "Variable-Motif-Deviation-Scores", 
        width = 12, height = 12, ArchRProj = cd4na_atac, addDOC = FALSE)

In [ ]:
plotVarDev

In [ ]:
saveArchRProject(ArchRProj = cd4na_atac)

### plot regions in Archr 

In [ ]:
source('/home/jupyter/github/scATAC_Supplements/ArchR_Export_ATAC_Data.R')

In [ ]:
# Extract fragments by populations from an ArchR Project: getPopFrags()
cd4na_Clusters_0.5_frags <- getPopFrags(cd4na_atac, 'Clusters_0.5',cellSubsets = 'ALL' ,
                                           region = NULL, numCores = 30, 
                        NormMethod = "nfrags", blackList = NULL, overlapList = 50)

In [ ]:
cd4na_Clusters_0.5_frags_files <- FragsToCoverageFiles(cd4na_Clusters_0.5_frags, files = "BigWig",
                                             genome = "hg38", fname = NULL, outDir = NULL, numCores=30)

In [ ]:
getAvailableMatrices(cd4na_atac)
cd4na_atac <- addMotifAnnotations(ArchRProj = cd4na_atac, motifSet = "JASPAR2020", name = "JasparMotifs")

In [ ]:
getAvailableMatrices(cd4na_atac)

In [ ]:
cd4na_Clusters_0.5_frags_PLCG2 <- getbpCounts(regionString='chr16:81,774,291-81,967,685', 
                                          popFrags=cd4na_Clusters_0.5_frags,
                                        numCores = 30, returnGRanges = FALSE)

In [ ]:
cd4na_Clusters_0.5_frags_PLCG2 %>% head()

In [ ]:
suppressMessages(p1 <- plotRegion(cd4na_Clusters_0.5_frags_PLCG2, 
                                  ArchRProj = cd4na_atac, motifSetName =  "JasparMotifs"))


In [ ]:
pdf(file.path(fig_path, paste0(proj_name, '_atac_Clusters_0.8_PLCG2_gene_coverage.pdf')), width = 12, height = 12)
suppressWarnings(p1)
dev.off()

In [ ]:
p1

In [ ]:
sessionInfo()